<a href="https://colab.research.google.com/github/PinganYANG/DL_learn/blob/main/%E9%81%97%E4%BC%A0%E5%9B%A0%E5%AD%90_positif_no_jit_version_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Librairy

In [43]:
# < 50w population
#


In [44]:

!pip install gplearn


In [45]:
import gplearn
print(gplearn.__version__)

0.4.2


In [46]:
# # 使用 Jupyter Notebook 的 magic command 来安装 scikit-learn
!pip install scikit-learn==1.5.2



In [47]:
from   gplearn            import genetic
from   gplearn.functions  import make_function
from   gplearn.genetic    import SymbolicTransformer, SymbolicRegressor
from   gplearn.fitness    import make_fitness
from   scipy              import stats
from   numba              import njit
from   scipy.stats        import spearmanr
from   scipy.stats        import rankdata
from   tqdm               import tqdm
import math
import time
import matplotlib.pyplot      as plt
import seaborn                as sns
import numpy                  as np
import pandas                 as pd
import numba                  as nb
import statsmodels.api        as sm
import sys
from   google.colab           import drive
from numpy.lib.stride_tricks import sliding_window_view
from numba import guvectorize
import numpy as np
pd.set_option('display.max_columns', None)

In [48]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [49]:
test_data = {'Col1': range(1, 201), 'Col2': range(401, 601), 'Col3': range(801, 1001), 'Col4': range(1201, 1401)}
df = pd.DataFrame(test_data)

In [50]:
sr  = np.array(df).T.flatten()

#读取数据

In [9]:

# # file_path = '/content/drive/MyDrive/Colab/遗传结果/selected_raw_df.paquet'
# data_new_cols = pd.read_parquet('/content/drive/MyDrive/Colab/selected_raw_df.paquet')


#data_new_cols

In [10]:
# data_new_cols['CDLDOJISTAR']

In [11]:
file_path = "/content/drive/MyDrive/Colab/data_clean_合集_最新.pkl"
data_clean=pd.read_pickle(file_path)
base_Factor = data_clean.columns.tolist()
# data_clean=pd.read_pickle(file_path)
data_clean=data_clean[['open_composed','high_composed','low_composed','close_composed','adjClose_composed','y']]

data_clean


open_composed  high_composed  low_composed  close_composed  \
ticker date                                                                     
0050   2010-01-05      -0.258004      -0.263814     -0.257305       -0.260749   
       2010-01-06      -0.258044      -0.263636     -0.257056       -0.260265   
       2010-01-07      -0.257671      -0.263533     -0.256978       -0.260577   
       2010-01-08      -0.257922      -0.263867     -0.257266       -0.260663   
       2010-01-11      -0.258031      -0.263707     -0.257002       -0.260414   
...                          ...            ...           ...             ...   
XSMI   2024-06-24       2.339524       2.359556      2.361499        2.374765   
       2024-06-25       2.362142       2.355507      2.378049        2.361970   
       2024-06-26       2.374585       2.359620      2.369213        2.349027   
       2024-06-27       2.361202       2.341013      2.369747        2.350958   
       2024-06-28       2.365084       2.342640      2.347145        2.326038   

                   adjClose_composed         y  
ticker date                                     
0050   2010-01-05          -0.265271 -0.011416  
       2010-01-06          -0.265076  0.004023  
       2010-01-07          -0.264911  0.001810  
       2010-01-08          -0.265171 -0.008158  
       2010-01-11          -0.265050 -0.015686  
...                              ...       ...  
XSMI   2024-06-24           2.676255 -0.007599  
       2024-06-25           2.661981 -0.007599  
       2024-06-26           2.647542 -0.007599  
       2024-06-27           2.649696 -0.007599  
       2024-06-28           2.621897 -0.007599  

[271800 rows x 6 columns]

In [12]:

data_clean =data_clean.copy()

## 加入新列

# 预处理

In [13]:


data_factor = data_clean.drop('y', axis=1)

data_factor

open_composed  high_composed  low_composed  close_composed  \
ticker date                                                                     
0050   2010-01-05      -0.258004      -0.263814     -0.257305       -0.260749   
       2010-01-06      -0.258044      -0.263636     -0.257056       -0.260265   
       2010-01-07      -0.257671      -0.263533     -0.256978       -0.260577   
       2010-01-08      -0.257922      -0.263867     -0.257266       -0.260663   
       2010-01-11      -0.258031      -0.263707     -0.257002       -0.260414   
...                          ...            ...           ...             ...   
XSMI   2024-06-24       2.339524       2.359556      2.361499        2.374765   
       2024-06-25       2.362142       2.355507      2.378049        2.361970   
       2024-06-26       2.374585       2.359620      2.369213        2.349027   
       2024-06-27       2.361202       2.341013      2.369747        2.350958   
       2024-06-28       2.365084       2.342640      2.347145        2.326038   

                   adjClose_composed  
ticker date                           
0050   2010-01-05          -0.265271  
       2010-01-06          -0.265076  
       2010-01-07          -0.264911  
       2010-01-08          -0.265171  
       2010-01-11          -0.265050  
...                              ...  
XSMI   2024-06-24           2.676255  
       2024-06-25           2.661981  
       2024-06-26           2.647542  
       2024-06-27           2.649696  
       2024-06-28           2.621897  

[271800 rows x 5 columns]

In [14]:
# base_Factor = data.columns.tolist()
# base_Factor0 = []
# for b in base_Factor:
#   if b == 'y':
#     pass
#   else:
#     base_Factor0.append(b)
# base_Factor = base_Factor0
# # base_Factor = selected_features

X_train_df = data_factor
y_train_df = data_clean['y']

x=X_train_df.to_numpy(dtype=np.float64)
y=y_train_df.to_numpy(dtype=np.float64)

x1_df = X_train_df[X_train_df.index.get_level_values(1) <  '2021-01-01']
x1_np=x1_df.to_numpy(dtype=np.float64)


x2_df = X_train_df[X_train_df.index.get_level_values(1) >= '2021-01-01']
x2_np=x2_df.to_numpy(dtype=np.float64)

y1_df = y_train_df[y_train_df.index.get_level_values(1) <  '2021-01-01']
y1_np = y1_df.to_numpy(dtype=np.float64)

y2_df=y_train_df[y_train_df.index.get_level_values(1) >= '2021-01-01']
y2_np=y2_df.to_numpy(dtype=np.float64)



In [15]:
window=200
window_list = [i for i in range(2, window)]

In [16]:
x1_df

open_composed  high_composed  low_composed  close_composed  \
ticker date                                                                     
0050   2010-01-05      -0.258004      -0.263814     -0.257305       -0.260749   
       2010-01-06      -0.258044      -0.263636     -0.257056       -0.260265   
       2010-01-07      -0.257671      -0.263533     -0.256978       -0.260577   
       2010-01-08      -0.257922      -0.263867     -0.257266       -0.260663   
       2010-01-11      -0.258031      -0.263707     -0.257002       -0.260414   
...                          ...            ...           ...             ...   
XSMI   2020-12-25       1.952384       1.943082      1.961980        1.951595   
       2020-12-28       1.951278       1.951266      1.971575        1.968849   
       2020-12-29       1.989741       1.986571      2.006750        1.998377   
       2020-12-30       2.009182       1.998489      2.021344        1.994144   
       2020-12-31       2.013310       2.003183      2.023579        2.001405   

                   adjClose_composed  
ticker date                           
0050   2010-01-05          -0.265271  
       2010-01-06          -0.265076  
       2010-01-07          -0.264911  
       2010-01-08          -0.265171  
       2010-01-11          -0.265050  
...                              ...  
XSMI   2020-12-25           1.979642  
       2020-12-28           1.997038  
       2020-12-29           2.027168  
       2020-12-30           2.022763  
       2020-12-31           2.030211  

[206352 rows x 5 columns]

#unstack

In [17]:
t_list = ['open','high','low','close','volume','adjClose','vwap','open_composed','high_composed','low_composed','close_composed','adjClose_composed','volume_composed','vwap_composed','returns','returns_composed','AVGPRICE','MEDPRICE','TYPPRICE','WCLPRICE','BOP','WILLR','SAR','TRANGE','HT_DCPERIOD','HT_DCPHASE','HT_PHASOR','ht_phasor_quadrature','HT_ SINE','ht_sine_leadsine','HT_TRENDMODE','MA_5','MA_10','MA_15','MA_20','ULTOSC','KAMA_5','KAMA_10','KAMA_15','KAMA_20','TEMA_5','TEMA_10','TEMA_15','TEMA_20','TRIMA_5','TRIMA_10','TRIMA_15','TRIMA_20','SMA_5','SMA_10','SMA_15','SMA_20','MACD','MACD_Signal','STOCH_%K_10','STOCH_%D_10','STOCH_%K_20','STOCH_%D_20','RSI_5','RSI_10','RSI_15','RSI_20','BB_MIDDLE','BB_UPPER','BB_LOWER','APO_short','APO_long','AROON_UP_10','AROON_DOWN_10','AROON_UP_25','AROON_DOWN_25','AROON_UP_50','AROON_DOWN_50','CMO_5','CMO_10','CMO_15','CMO_20','DX_5','DX_10','DX_15','DX_20','MOM_5','MOM_10','MOM_15','MOM_20','PLUS_DM_5','PLUS_DM_10','PLUS_DM_15','PLUS_DM_20','ROC_5','ROC_10','ROC_15','ROC_20','ROCP_5','ROCP_10','ROCP_15','ROCP_20','ROCR_5','ROCR_10','ROCR_15','ROCR_20','TRIX_5','TRIX_10','TRIX_15','TRIX_20','ADX_5','ADX_10','ADX_15','ADX_20','ADXR_5','ADXR_10','ADXR_15','ADXR_20','ATR_5','ATR_10','ATR_15','ATR_20','AROONOSC','AROONOSC_composed','WR','WR_composed','CCI','CCI_composed','ATR','ATR_composed','NATR','NATR_composed','ADX','ADX_composed','MFI','MFI_composed']

In [18]:
nombre_ticker=len(x1_df.index.get_level_values(0).unique())
print(f'nombre_ticker:{nombre_ticker}')

nombre_date  =len(x1_df.index.get_level_values(1).unique())
print(nombre_date)

nombre_ticker:72
2866


In [19]:
def functions_name_arity1(dict_functions):
    functions = []
    names = []
    for name, func in dict_functions.items():
        custom_func = make_function(function=func, name=name, arity=1)
        functions.append(custom_func)
        names.append(name)
    return functions, names

def functions_name_arity2(dict_functions):
    functions = []
    names = []
    for name, func in dict_functions.items():
        custom_func = make_function(function=func, name=name, arity=2)
        functions.append(custom_func)
        names.append(name)
    return functions, names

In [20]:
def spearman_corr(x, y):

    x_rank = np.argsort(np.argsort(x))
    y_rank = np.argsort(np.argsort(y))

    correlation_matrix = np.corrcoef(x_rank, y_rank)
    return correlation_matrix[0, 1]

def rankic_normal(y,y_pred, w):
    if len(y) < nombre_ticker or len(y_pred) < nombre_ticker:
        return -float('inf')
    window = 200 # 举个例子
    num_rows_to_delete = window * 5

    x1_unstack = y.reshape(nombre_ticker, -1).T
    x1_unstack = np.delete(x1_unstack, slice(0, num_rows_to_delete), axis=0)
    x2_unstack = y_pred.reshape(nombre_ticker, -1).T
    x2_unstack = np.delete(x2_unstack, slice(0, num_rows_to_delete), axis=0)

    correlations = []
    for i in range(x1_unstack.shape[0]):
        correlation = spearman_corr(x1_unstack[i], x2_unstack[i])
        correlations.append(correlation)

    mean_correlation = np.mean(correlations)
    return mean_correlation



rankic_normal = make_fitness(function=rankic_normal, greater_is_better=True) # 正反方向

####填充

In [21]:

#填充0 和nan

def forward_fill(arr):
    # 创建布尔掩码，找出 NaN 值的位置
    mask = np.isnan(arr)

    # 遍历数组并对 NaN 值进行填充
    for i in range(1, len(arr)):
        if mask[i]:  # 如果当前值是 NaN
            arr[i] = arr[i - 1]  # 用前一个非 NaN 值填充
    return arr


def backward_fill(arr):
    # 创建布尔掩码，找出 NaN 值的位置
    mask = np.isnan(arr)

    # 遍历数组并对 NaN 值进行向后填充
    for i in range(len(arr) - 2, -1, -1):  # 从倒数第二个元素开始向前遍历
        if mask[i]:  # 如果当前值是 NaN
            arr[i] = arr[i + 1]  # 用后一个非 NaN 值填充
    return arr

def handle_nan(X):
    # 前值填充
    X = np.copy(X)
    _temp = np.nan
    na_len = 0
    for i in range(len(X)):
        if np.isnan(X[i]):
            X[i] = _temp
            na_len += 1
        else:
            _temp = X[i]
    return X, na_len


# (nb.types.Tuple((nb.float64[:], nb.int64))(nb.float64[:]))
# def handle_nan2(X):
#     # 数组拷贝
#     X = np.copy(X)
#     na_len = 0
#     for j in range(X.shape[1]):  # 迭代每一列
#         _temp = np.nan
#         for i in range(X.shape[0]):  # 迭代每一列的每一行
#             if np.isnan(X[i, j]):
#                 X[i, j] = _temp
#                 na_len += 1
#             else:
#                 _temp = X[i, j]
#     return X, na_len

####双变量

In [22]:

def add(x1, x2):
    return x1 + x2


def sub(x1, x2):
    return x1 - x2


def mul(x1, x2):
    return x1 * x2


def div(x1, x2):
    if len(x1) <= 10:
       return np.ones_like(x1)
    if len(x2) <= 10:
       return np.ones_like(x2)

    result = np.where(np.abs(x2) > 0, np.divide(x1, x2), 1.)
    result = np.where(result == 0, np.nan, result)    #因为一的对数是0
    result = forward_fill(result)  # 先向前填充
    result = backward_fill(result)  # 然后向后填充，确保所有 NaN 都被处理
    return result


def max(x1, x2):
    return np.maximum(x1, x2)


def min(x1, x2):
    return np.minimum(x1, x2)


def _andpn(x1, x2):
    return np.where((x1 > 0) & (x2 > 0), 1.0, -1.0)


def _orpn(x1, x2):
    return np.where((x1 > 0) | (x2 > 0), 1.0, -1.0)


def _ltpn(x1, x2):
    return np.where(x1 < x2, 1.0, -1.0)


def _gtpn(x1, x2):
    return np.where(x1 > x2, 1.0, -1.0)


def _andp(x1, x2):
    return np.where((x1 > 0) & (x2 > 0), 1.0, 0.0)


def _orp(x1, x2):
    return np.where((x1 > 0) | (x2 > 0), 1.0, 0.0)


def _ltp(x1, x2):
    return np.where(x1 < x2, 1.0, 0.0)


def _gtp(x1, x2):
    return np.where(x1 > x2, 1.0, 0.0)


def _andn(x1, x2):
    return np.where((x1 > 0) & (x2 > 0), -1.0, 0.0)


def _orn(x1, x2):
    return np.where((x1 > 0) | (x2 > 0), -1.0, 0.0)


def _ltn(x1, x2):
    return np.where(x1 < x2, -1.0, 0.0)


def _gtn(x1, x2):
    return np.where(x1 > x2, -1.0, 0.0)


def _if(x1, x2, x3):
    # Assuming x2 and x3 are already float64 or at least compatible types
    return np.where(x1 > 0, x2, x3)


def _clear_by_cond(x1, x2, x3):
    """if x1 < x2 (keep NaN if and only if both x1 and x2 are NaN), then 0, else x3"""
    return np.where(x1 < x2, 0, np.where(~np.isnan(x1) | ~np.isnan(x2), x3, np.nan))

def _if_then_else(x1, x2, x3):
    """if x1 is nonzero (keep NaN), then x2, else x3"""
    return np.where(x1, x2, np.where(~np.isnan(x1), x3, np.nan))

def _if_cond_then_else(x1, x2, x3, x4):
    """if x1 < x2 (keep NaN if and only if both x1 and x2 are NaN), then x3, else x4"""
    return np.where(x1 < x2, x3, np.where(~np.isnan(x1) | ~np.isnan(x2), x4, np.nan))


####单变量

In [23]:

def sign(x):
    return np.sign(x)


def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def squre(x):
    return np.square(x)


def sqrt(x):
    return np.sqrt(np.abs(x))


def log(x):
    if len(x) <= 10:
       return np.ones_like(x)

    result = np.where(np.abs(x) > 0, np.log(np.abs(x)), np.nan) #等于0 的都 被设置为nan
    result = np.where(result == 0, np.nan, result)    #因为一的对数是0
    result = forward_fill(result)  # 先向前填充
    result = backward_fill(result)  # 然后向后填充，确保所有 NaN 都被处理
    return result


def inv(x):
    if len(x) <= 10:
       return np.ones_like(x)
    result = np.where(np.abs(x) > 0, 1. / x, np.nan)
    result = np.where(result == 0, np.nan, result)    #因为一的对数是0
    result = forward_fill(result)  # 先向前填充
    result = backward_fill(result)  # 然后向后填充，确保所有 NaN 都被处理
    return result


def sin(x):
    if len(x) <= 10:
       return np.ones_like(x)
    result = np.where(x == 0, np.nan, x)
    result = forward_fill(result)  # 先向前填充
    result = backward_fill(result)
    return np.sin(result)


def cos(x):
    if len(x) <= 10:
       return np.ones_like(x)
    result = np.where(x == 0, np.nan, x)
    result = forward_fill(result)  # 先向前填充
    result = backward_fill(result)
    return np.cos(result)



def tan(x):
    return np.tan(x)


def abs(x):
    return np.abs(x)


def neg(x):
    return np.negative(x)


def sum(x):
    if len(x) <= 10:
        return np.ones(x.size)  # 使用 x.size 确保返回的是一维数组
    x = x.ravel()
    x = x.reshape(nombre_ticker, nombre_date).T
    data_new = np.empty_like(x)

    for i in range(x.shape[0]):
        row_sum = np.sum(x[i, :])  # 计算当前行的总和
        data_new[i, :] = row_sum   # 将每个元素都设置为行总和
    return data_new.T.flatten()


def scale(sr):
    k = 1  # 常数 k
    abs_sum = np.sum(np.abs(sr))  # 计算绝对值的总和
    if abs_sum == 0:
        return np.zeros_like(sr, dtype=np.float64)  # 防止除以 0 的情况，明确返回浮点数数组
    return (sr * k) / abs_sum  # 返回缩放后的序列


def _delayy(x):
    # 创建一个只包含 np.nan 的 NumPy 数组
    nan_array = np.array([np.nan], dtype=x.dtype)
    result = np.concatenate((nan_array, x[:-1]))
    return np.nan_to_num(result, nan=0)




def _delta(x):
    _ = np.nan_to_num(x, nan=0)
    return _ - np.nan_to_num(_delayy(_), nan=0)


def _signedpower(x):
    _ = np.nan_to_num(x, nan=0)
    return np.sign(_) * (abs(_) ** 2)



def _stdd(x):
    return np.array([np.std(x)] * len(x))

def _rankk(x):
    return x.argsort().astype(np.float64)

(nb.float64[:](nb.float64[:],nb.float64[:]))
def _combine(X, Y):
    p1 = 15485863
    p2 = 32416190071
    p3 = 100000007
    return np.mod(X * p1 + Y * p2, p3).astype(np.float64)


def Rank_original(sr):
    n = len(sr)
    result = np.empty(n)

    sorted_indices = np.argsort(sr)
    ranks = np.empty_like(sorted_indices)

    rank = 1
    for idx in sorted_indices:
        ranks[idx] = rank
        rank += 1

    result[:] = ranks / n

    return result


def Rank_rows(X):
    if len(X) <= 10:
       return np.ones_like(X)
    X = X.ravel()
    X = X.reshape(nombre_ticker, nombre_date).T
    num_rows= X.shape[0]
    result = np.empty_like(X)

    # 对每一行进行排名处理
    for row in range(num_rows):
        sr = X[row, :]  # 提取当前行
        n = len(sr)
        sorted_indices = np.argsort(sr)
        ranks = np.empty_like(sorted_indices)

        rank = 1
        for idx in sorted_indices:
            ranks[idx] = rank
            rank += 1

        # 将排名规范化并保存到结果数组的相应行
        result[row, :] = ranks / n

    return result.T.flatten()


def custom_diff(arr):
    return np.array([arr[i] - arr[i-1] for i in range(1, len(arr))])


In [24]:

def cumulative_rank(X):
    if len(X) <= 10:
        return np.zeros_like(X)

    # 重塑数组为期望的形状（3行4列），然后转置成4行3列
    X = X.ravel()
    X = X.reshape(nombre_ticker,nombre_date).T
    num_columns = X.shape[1]  # 列数
    num_rows = X.shape[0]  # 行数
    result = np.zeros_like(X)  # 为排名结果创建一个同样形状的空数组

    # 对每一列进行操作
    for col in range(num_columns):
        # 对整列进行初始排名
        full_sorted_indices = np.argsort(X[:, col])  # 获取当前列的升序索引
        full_ranks = np.argsort(full_sorted_indices) + 1  # 通过再次排序索引得到排名
        result[:, col] = full_ranks  # 存储初始排名

        # 逐步减少包含的元素，更新排名
        for i in range(num_rows - 1, 0, -1):  # 从倒数第二个元素开始至第一个元素
            current_segment = X[:i, col]  # 取当前列的前i个元素
            sorted_indices = np.argsort(current_segment)  # 对这部分进行排序，得到索引
            current_ranks = np.argsort(sorted_indices) + 1  # 再次对索引排序，得到排名

            # 用更短段的新排名更新原排名，但只更新至当前考虑的长度
            result[:i, col] = current_ranks  # 这样可以确保使用最新的、基于最短段的排名

    return result.T.flatten()

####横向变量

In [25]:



def normalize_ranks(ranks):
  # checked
    normalized_ranks = np.empty_like(ranks, dtype=np.float64)

    # 找到每行的最大值并归一化
    for i in range(nombre_date):
        max_rank = np.max(ranks[i, :])  # 找到每行的最大值
        for j in range(nombre_ticker):
            normalized_ranks[i, j] = ranks[i, j] / max_rank  # 归一化当前行

    return normalized_ranks


def z_score(x):
  # checked
    if len(x) <= 10:
       return np.ones_like(x)
    x = x.ravel()
    x = x.reshape(nombre_ticker,nombre_date).T

    row_means = np.empty((x.shape[0], x.shape[1]))

    for i in range(x.shape[0]):
        sum_row = 0.0
        for j in range(x.shape[1]):
            sum_row += x[i, j]
        row_means[i] = sum_row / x.shape[1] #每行的总和除以列出

    x_sub_row_means = x - row_means

    row_stds = np.empty_like(x)
    for i in range(x.shape[0]):
        sum_squared_diff = 0.0
        for j in range(x.shape[1]):
            sum_squared_diff += (x_sub_row_means[i, j]) ** 2
        std = np.sqrt(sum_squared_diff / x.shape[1])
        row_stds[i, :] = std  # 每一行填充标准差
    row_stds_rounded = np.round(row_stds, 3)


    data_pivot=x_sub_row_means/row_stds_rounded

    return data_pivot.T.flatten()

#### 第一组

In [26]:
def all_function_0(window):  ####
    # checked
    # def RETURN(X):
    #     if len(X) <= 10:
    #           return np.ones_like(X)

    #     X = X.reshape(nombre_ticker, nombre_date).T
    #     result = np.empty_like(X)

    #     # result = np.empty(len(X))
    #     # result[0] = np.nan
    #     for i in range(window-1, X.shape[0]):
    #         if X[i - window] == 0:
    #             result[i] = np.nan
    #         else:
    #             result[i] = (X[i] / X[i - window]) - 1
    #     return result.T.flatten()
    # RETURN.__name__ = f"RETURN_{window}"
    # return RETURN

    def RETURN(X):
        """
        Calculates the RETURN factor in 2D format.

        Args:
            X: Input data as a 1D numpy array.
            window: Window size for calculation.
            nombre_ticker: Number of tickers.
            nombre_date: Number of dates.

        Returns:
            The RETURN factor in 2D format as a numpy array.
        """
        if len(X) <= 10:
            return np.ones_like(X)

        # Reshape the input data to 2D (nombre_ticker, nombre_date)
        X = X.reshape(nombre_ticker, nombre_date).T

        # Use sliding_window_view to create the sliding windows
        windows = sliding_window_view(X, window, axis=0)

        # Calculate the return factor for each window
        result = np.full_like(X, np.nan, dtype=np.float64)

        result[window-1:, :] = (windows[:, :, -1] / windows[:, :, 0]) - 1
        result[:window-1,:] = result[window-1,:]
        del X, windows
        return result.T.flatten()


    RETURN.__name__ = f"RETURN_{window}"
    return RETURN

dict_functions = {f"RETURN_{i}": all_function_0(i) for i in window_list}

RETURN_functions, RETURN_names = functions_name_arity1(dict_functions)



def all_function_1(window):      #### 整体向后错window 个位置 比如：1 2 3 4 5 6 变成 1 1 1 1 2 3
    # def delay(sr):
    #     result = np.empty_like(sr)
    #     if window > 0:
    #         result[:window] = sr[0]  # 使用输入数组的第一个值填充前 window 个值
    #         result[window:] = sr[:-window]
    #     else:
    #         result[:] = sr[:]  # window为0时直接复制数组
    #     return result
    # delay.__name__ = f"delay_{window}"
    # return delay
    #### 矩阵形式
    # def delay(X):
    #     if len(X) <= 10:
    #         return np.ones_like(X)
    #     X = X.reshape(nombre_ticker, nombre_date).T  # 将矩阵转置，便于逐列操作
    #     result = np.empty_like(X,dtype=np.float64)

    #     for j in range(X.shape[1]):  # 对每一列（每个序列）分别处理
    #         sr = X[:, j]
    #         if window > 0:
    #             result[:window, j] = sr[0]  # 前 `window` 个值用第一个值填充
    #             result[window:, j] = sr[:-window]  # 剩余位置进行延迟填充
    #         else:
    #             result[:, j] = sr[:]  # 如果 `window` 为 0，直接复制数组

    #     return result.T.flatten()  # 转置并扁平化结果

    def delay(X):
        """
        Calculates the delay factor in 2D format.

        Args:
            X: Input data as a 1D numpy array.
            window: Window size for calculation.
            nombre_ticker: Number of tickers.
            nombre_date: Number of dates.

        Returns:
            The delay factor in 2D format as a numpy array.
        """
        if len(X) <= 10:
            return np.ones_like(X)

        # Reshape the input data to 2D (nombre_ticker, nombre_date)
        X = X.reshape(nombre_ticker, nombre_date).T

        # Initialize the result array with NaN
        result = np.full_like(X, np.nan, dtype=np.float64)

        # Handle window > 0 case
        if window > 0:
            # Use sliding window view to shift the data
            windows = sliding_window_view(X, window, axis=0)

            # Shift values by the window size
            result[window-1:, :] = windows[:, :, 0]

            # Fill initial values with the first value of each column
            result[:window, :] = X[0, :]

        else:
            # If window is 0, copy the column directly
            result[:] = X
        del X, windows
        return result.T.flatten()

    delay.__name__ = f"delay_{window}"
    return delay

dict_functions = {f"delay_{i}": all_function_1(i) for i in window_list}
delay_functions, delay_names = functions_name_arity1(dict_functions)



def all_function_02(window):
    # checked
    def Delta2v(X1, X2):
        if len(X1) <= 10 or len(X2) <= 10:
            return np.ones_like(X1)

        # Reshape arrays to 2D (nombre_ticker, nombre_date)
        X1 = X1.reshape(nombre_ticker, nombre_date).T
        X2 = X2.reshape(nombre_ticker, nombre_date).T

        # Initialize result array with NaN values
        res = np.full_like(X1, np.nan, dtype=np.float64)

        # Calculate the difference with vectorized operation
        res[window-1:] = X1[window-1:] - X2[:-window+1]
        res[:window,:] = res[window-1,:]

        del X1, X2
        return res.T.flatten()

    Delta2v.__name__ = f"Delta2v_{window}"
    return Delta2v

dict_functions = {f"Delta2v_{i}": all_function_02(i) for i in window_list}
Delta2v_functions, Delta2v_names = functions_name_arity2(dict_functions)


def all_function_4(window):

    def Cov(x, y):
        if len(x) <= 10 or len(y) <= 10:
            return np.ones_like(x)

        # Reshape the input data to 2D [date, ticker]
        x = x.reshape(nombre_ticker, nombre_date).T
        y = y.reshape(nombre_ticker, nombre_date).T

        # Use sliding window view to generate windows for x and y
        x_windows = sliding_window_view(x, window, axis=0)
        y_windows = sliding_window_view(y, window, axis=0)

        # Calculate means for window data using broadcasting
        x_mean = np.mean(x_windows, axis=2, keepdims=True)
        y_mean = np.mean(y_windows, axis=2, keepdims=True)

        # Calculate covariance using broadcasting
        numerator = np.sum((x_windows - x_mean) * (y_windows - y_mean), axis=2)
        denominator = window - 1

        # Avoid division by zero
        denominator = np.where(denominator > 0.001, denominator, 0.001)

        # Calculate correlation using covariance / sqrt(variance_x * variance_y)
        covariance = numerator / denominator

        # Initialize result matrix and assign covariance values
        result = np.full_like(x, np.nan, dtype=np.float64)
        result[window - 1:, :] = covariance  # Assign values after the first full window
        result[:window, :] = result[window-1, :]
        # print(result.shape)
        # print(result[window-1, :])
        del x, y, x_windows, y_windows, x_mean, y_mean, numerator, denominator, covariance
        return result.T.flatten()  # Flatten back to 1D


    Cov.__name__ = f"Cov_{window}"
    return Cov

dict_functions = {f"Cov_{i}": all_function_4(i) for i in window_list}
Cov_functions, Cov_names = functions_name_arity2(dict_functions)

def all_function_6(window):
    # @njit(nb.float64[:](nb.float64[:]))
    # def Prod(sr):
    #     n = len(sr)
    #     result = np.empty(n)
    #     if n > 0:
    #         initial_prod = sr[0]  # 这里不使用sr[0]**window以避免对0的任何次方产生不适当的结果
    #         result[:window-1] = initial_prod
    #         for i in range(window - 1, n):
    #             result[i] = np.prod(sr[i - window + 1:i + 1])
    #     else:
    #         result[:] = 1
    #     return result

    ##### 矩阵形式
    def Prod(X):
        if len(X) <= 10:
            return np.ones_like(X)

        # Reshape the input data to 2D (nombre_ticker, nombre_date)
        X = X.reshape(nombre_ticker, nombre_date).T

        # Initialize the result array with NaN values
        result = np.full_like(X, np.nan, dtype=np.float64)

        # Use sliding_window_view to create the sliding windows for each column
        windows = sliding_window_view(X, window, axis=0)

        # Calculate the product for each window and assign it to the result array
        result[window-1:] = np.prod(windows, axis=2)

        # Fill the first `window-1` elements with the initial value (first element in each column)
        result[:window-1] = X[0, :]
        del X, windows
        return result.T.flatten()

    Prod.__name__ = f"Prod_{window}"
    return Prod

dict_functions = {f"Prod_{i}": all_function_6(i) for i in window_list}
Prod_functions, Prod_names = functions_name_arity1(dict_functions)


def all_function_8(window):
  # to condiser
    # def Tsrank(sr):
    #     n = len(sr)
    #     result = np.empty(n)
    #     if n > 0:
    #         result[:window-1] = 1
    #         for i in range(window - 1, n):
    #             window_data = sr[i - window + 1:i + 1]
    #             sorted_indices = np.argsort(window_data)
    #             ranks = np.empty(len(window_data))
    #             ranks[sorted_indices] = np.arange(1, len(window_data) + 1)
    #             result[i] = ranks[-1]  # 最后一个元素的排名
    #     else:
    #         result[:] = 0
    #     return result

    #### 矩阵形式
    def Tsrank_col(X):
        if len(X) <= 10:
            return np.ones_like(X)

        # Reshape the input data to 2D (nombre_ticker, nombre_date)
        X = X.reshape(nombre_ticker, nombre_date).T

        # Initialize the result array with NaN values
        result = np.full_like(X, np.nan, dtype=np.float64)

        # Set initial window values to 1
        result[:window - 1, :] = 1

        # Use sliding window view to generate windows
        windows = sliding_window_view(X, window, axis=0)

        # Find the rank of the maximum value in each window
        max_indices = np.argmax(windows, axis=2)  # Find the index of the maximum value in each window

        # Assign the rank (max index + 1 for 1-based rank) to the result
        result[window - 1:, :] = max_indices + 1  # Rank of the maximum value

        del X, windows, max_indices
        return result.T.flatten()


    Tsrank_col.__name__ = f"Tsrank_col_{window}"
    return Tsrank_col

dict_functions = {f"Tsrank_col_{i}": all_function_8(i) for i in window_list}
Tsrank_col_functions , Tsrank_col_names= functions_name_arity1(dict_functions)


def all_function_11(window):


    # def Lowday(sr):
    #     n = len(sr)
    #     result = np.empty(n)
    #     if n > 0 and not np.isnan(sr[0]):
    #         result[:window-1] = sr[0]
    #     else:
    #         result[:window-1] = 0
    #     for i in range(window - 1, n):
    #         if np.all(np.isnan(sr[i - window + 1:i + 1])):
    #             result[i] = 0
    #         else:
    #             window_data = sr[i - window + 1:i + 1]
    #             min_index = np.argmin(window_data)
    #             result[i] = window - min_index

    #     return result


    # Lowday.__name__ = f"Lowday_{window}"
    # return Lowday

    ##### 矩阵形式
    def Lowday(X):
        if len(X) <= 10:
            return np.ones_like(X)

        # Reshape the input data to 2D (nombre_ticker, nombre_date)
        X = X.reshape(nombre_ticker, nombre_date).T

        # Initialize the result array with NaN values
        result = np.full_like(X, np.nan, dtype=np.float64)

        # Handle the initial window values
        result[:window-1, :] = X[0, :]

        # Use sliding window view to generate windows of data
        windows = sliding_window_view(X, window, axis=0)

        # Calculate the index of the minimum value in each window
        min_indices = np.argmin(windows, axis=2)

        # Calculate the offset (window size - min index) and assign it to the result
        result[window-1:, :] = window - min_indices
        result[:window-1,:] = result[window-1,:]

        del X, windows, min_indices
        return result.T.flatten()


    Lowday.__name__ = f"Lowday_{window}"
    return Lowday


dict_functions = {f"Lowday_{i}": all_function_11(i) for i in window_list}
Lowday_functions , Lowday_names  = functions_name_arity1(dict_functions)



def all_function_12(window):


    # def Highday(sr):
    #     n = len(sr)
    #     result = np.empty(n)
    #     if n > 0 and not np.isnan(sr[0]):
    #         result[:window-1] = sr[0]
    #     else:
    #         result[:window-1] = 0
    #     for i in range(window - 1, n):
    #         if np.all(np.isnan(sr[i - window + 1:i + 1])):
    #             result[i] = 0
    #         else:
    #             window_data = sr[i - window + 1:i + 1]
    #             max_index = np.argmax(window_data)
    #             result[i] = window - max_index

    #     return result


    # Highday.__name__ = f"Highday_{window}"
    # return Highday

    ##### 矩阵形式
    def Highday(X):
        if len(X) <= 10:
            return np.ones_like(X)

        # Reshape the input data to 2D (nombre_ticker, nombre_date)
        X = X.reshape(nombre_ticker, nombre_date).T

        # Initialize the result array with NaN values
        result = np.full_like(X, np.nan, dtype=np.float64)

        # Handle the initial window values
        result[:window-1, :] = X[0, :]

        # Use sliding window view to generate windows of data
        windows = sliding_window_view(X, window, axis=0)

        # Calculate the index of the maximum value in each window
        max_indices = np.argmax(windows, axis=2)

        # Calculate the offset (window size - max index) and assign it to the result
        result[window-1:, :] = window - max_indices
        result[:window-1,:] = result[window-1,:]

        del X, windows, max_indices
        return result.T.flatten()



    Highday.__name__ = f"Highday_{window}"
    return Highday

dict_functions = {f"Highday_{i}": all_function_12(i) for i in window_list}
Highday_functions , Highday_names = functions_name_arity1(dict_functions)



def all_function_13(window):
    # def Decaylinear(sr):

    #     n = len(sr)
    #     result = np.empty(n)
    #     weights = np.arange(1, window + 1)
    #     sum_weights = np.sum(weights)
    #     if n > 0 and not np.isnan(sr[0]):
    #         result[:window-1] = sr[0]
    #     else:
    #         result[:window-1] = 0  # 如果第一个值是NaN或数组为空，选择填充0或其他值
    #     for i in range(window - 1, n):
    #         window_data = sr[i - window + 1:i + 1]
    #         if np.all(np.isnan(window_data)):
    #             result[i] = 0  # 如果窗口数据全为 NaN，则赋值为0
    #         else:
    #             result[i] = np.sum(weights * window_data) / sum_weights

    #     return result

    ##### 矩阵形式
    def Decaylinear(X):
      if len(X) <= 10:
          return np.ones_like(X)

      # Reshape the input data to 2D (nombre_ticker, nombre_date)
      X = X.reshape(nombre_ticker, nombre_date).T

      # Initialize the result array
      result = np.empty_like(X, dtype=np.float64)

      # Create the weights array (linear increasing weights)
      weights = np.arange(1, window + 1)
      sum_weights = np.sum(weights)  # Sum of the weights

      # Use sliding window view to create the sliding windows
      windows = sliding_window_view(X, window, axis=0)

      # Apply the weighted average calculation for each window
      weighted_sums = np.sum(windows * weights, axis=2)  # Element-wise multiplication and summing across the window axis

      # Calculate the weighted average
      result[window-1:] = weighted_sums / sum_weights

      # Handle the first `window-1` elements (initialize with the first value or zero)
      result[:window-1,:] = result[window-1,:]


      del X, sum_weights, weights, windows, weighted_sums
      return result.T.flatten()




    Decaylinear.__name__ = f"Decaylinear_{window}"
    return Decaylinear

dict_functions = {f"Decaylinear_{i}": all_function_13(i) for i in window_list}
Decaylinear_functions , Decaylinear_names = functions_name_arity1(dict_functions)


def all_function_14(window):
    # def Wma(sr):
    #     n = len(sr)
    #     result = np.empty(n)
    #     weights = np.power(0.9, np.arange(window - 1, -1, -1))
    #     sum_weights = np.sum(weights)
    #     if n > 0 and not np.isnan(sr[0]):
    #         result[:window-1] = sr[0]
    #     else:
    #         result[:window-1] = 0  # 如果第一个值是NaN或数组为空，选择填充0或其他值
    #     for i in range(window - 1, n):
    #         window_data = sr[i - window + 1:i + 1]
    #         if np.all(np.isnan(window_data)):
    #             result[i] = 0  # 如果窗口数据全为 NaN，则赋值为0
    #         else:
    #             result[i] = np.sum(weights * window_data) / sum_weights

    #     return result

    ##### 矩阵形式
    def Wma(X):
        if len(X) <= 10:
            return np.ones_like(X)

        # Reshape the input data to 2D (nombre_ticker, nombre_date)
        X = X.reshape(nombre_ticker, nombre_date).T

        # Initialize the result array
        result = np.empty_like(X, dtype=np.float64)

        # Generate the weights (decreasing weights)
        weights = np.power(0.9, np.arange(window - 1, -1, -1))  # Weights decrease exponentially
        sum_weights = np.sum(weights)  # Sum of the weights

        # Use sliding window view to generate the sliding windows of data
        windows = sliding_window_view(X, window, axis=0)

        # Apply the weighted average calculation for each window
        weighted_sums = np.sum(windows * weights, axis=2)  # Element-wise multiplication and summing across the window axis
        result[window-1:] = weighted_sums / sum_weights

        # Handle the first `window-1` elements (initialize with the first valid value or zero)
        for j in range(X.shape[1]):
            first_non_nan = np.nan
            for i in range(window - 1, -1, -1):
                if not np.isnan(X[i, j]):
                    first_non_nan = X[i, j]
                    break
            if not np.isnan(first_non_nan):
                result[:window, j] = first_non_nan
            else:
                result[:window, j] = 0  # If all values are NaN, fill with 0
        del X, weights, sum_weights, windows, weighted_sums
        return result.T.flatten()


    Wma.__name__ = f"Wma_{window}"
    return Wma

dict_functions = {f"Wma_{i}": all_function_14(i) for i in window_list}
Wma_functions , Wma_names = functions_name_arity1(dict_functions)





def all_function_16(window):
  # 不懂，等价于sum？
    # def Count(cond):
    #     n = len(cond)
    #     result = np.empty(n)  # 创建一个与输入长度相同的结果数组
    #     if n > 0:
    #         initial_count = cond[0] * window
    #         result[:window-1] = initial_count
    #         for i in range(window - 1, n):
    #             result[i] = np.sum(cond[i - window + 1:i + 1])  # 计算当前窗口的总和
    #     else:
    #         result[:] = 0
    #     return result

    #### 矩阵形式
    def Count(cond):
        if len(cond) <= 10:
            return np.ones_like(cond)

        # Reshape the input data to 2D (nombre_ticker, nombre_date)
        cond = cond.reshape(nombre_ticker, nombre_date).T

        # Initialize the result array
        result = np.empty_like(cond, dtype=np.float64)

        # Use sliding_window_view to create sliding windows of data
        windows = sliding_window_view(cond, window, axis=0)

        # Calculate the sum of each window
        window_sums = np.sum(windows, axis=2)

        # Assign the window sums to the result for each window after the first
        result[window-1:] = window_sums

        # Handle the first `window-1` elements (initialize with a constant value based on the first column)
        result[:window-1] = cond[0, :] * window
        del cond, windows, window_sums
        return result.T.flatten()

    Count.__name__ = f"Count_{window}"
    return Count

dict_functions = {f"Count_{i}": all_function_16(i) for i in window_list}
Count_functions , Count_names= functions_name_arity1(dict_functions)

def all_function_17(window):
    def Regbeta(sr):
        n = len(sr)
        result = np.zeros(n)
        x = np.arange(1, window + 1)
        sum_x = np.sum(x)
        sum_x2 = np.sum(x * x)

        # Handle the first `window` positions by assigning the first non-NaN value
        first_non_nan = sr[0]
        for i in range(n):
            if not np.isnan(sr[i]):
                first_non_nan = sr[i]
                break
        result[:window] = first_non_nan

        if n >= window:
            # Use sliding_window_view to create the sliding windows
            windows = sliding_window_view(sr, window)

            # Calculate sums for each window
            sum_y = np.sum(windows, axis=1)
            sum_xy = np.sum(windows * x, axis=1)

            # Calculate beta for each window
            numerator = window * sum_xy - sum_x * sum_y
            denominator = window * sum_x2 - sum_x * sum_x

            # Avoid division by zero
            valid_denominator = denominator != 0
            result[window - 1:] = np.where(valid_denominator, numerator / denominator, 0)


            del windows, sum_y, sum_xy, numerator, denominator, valid_denominator
        return result

    # #### 矩阵形式
    # def Regbeta(X):
    #     if len(X) <= 10:
    #         return np.ones_like(X)
    #     X = X.reshape(nombre_ticker, nombre_date).T  # 转置矩阵，使每列是一个独立序列
    #     result = np.zeros_like(X)  # 初始化结果矩阵
    #     x = np.arange(1, window + 1)  # 时间序列
    #     sum_x = np.sum(x)
    #     sum_x2 = np.sum(x * x)
    #     denominator = window * sum_x2 - sum_x * sum_x  # 分母（相同窗口长度下为常数）

    #     for j in range(X.shape[1]):  # 对每一列（每个序列）分别处理
    #         sr = X[:, j]
    #         n = len(sr)

    #         # 初始化前 `window` 个位置
    #         first_non_nan = np.nan
    #         for i in range(n):
    #             if not np.isnan(sr[i]):
    #                 first_non_nan = sr[i]
    #                 break


    #         # 计算滑动窗口的线性回归斜率
    #         if n >= window and denominator != 0:
    #             for i in range(window - 1, n):
    #                 y = sr[i - window + 1:i + 1]  # 当前窗口的数据
    #                 if np.all(np.isnan(y)):
    #                     result[i, j] = 0  # 如果窗口数据全为 NaN，则赋值为0
    #                 else:
    #                     y = np.nan_to_num(y, nan=0.0)  # 将 NaN 转换为 0 以便计算
    #                     sum_y = np.sum(y)
    #                     sum_xy = np.sum(x * y)
    #                     numerator = window * sum_xy - sum_x * sum_y
    #                     result[i, j] = numerator / denominator  # 计算并存储 beta 值

    #         if not np.isnan(first_non_nan):
    #             result[:window, j] = first_non_nan  # 用第一个非 NaN 值填充前 `window` 个位置
    #         else:
    #             result[:window, j] = 0  # 如果全是 NaN，填充为 0

    #     return result.T.flatten()  # 转置并扁平化结果

    Regbeta.__name__ = f"Regbeta_{window}"
    return Regbeta



dict_functions = {f"Regbeta_{i}": all_function_17(i) for i in window_list}
Regbeta_functions , Regbeta_names= functions_name_arity1(dict_functions)


def all_function_18(window):
    # def ts_argmin(sr):
    #     n = len(sr)
    #     result = np.empty(n)  # 创建一个结果数组
    #     first_non_nan = sr[0]
    #     for i in range(n):
    #         if not np.isnan(sr[i]):
    #             first_non_nan = sr[i]
    #             break
    #     result[:window] = first_non_nan
    #     for i in range(window - 1, n):
    #         window_data = sr[i - window + 1:i + 1]  # 获取当前窗口的数据
    #         min_idx = np.argmin(window_data)  # 计算最小值的索引
    #         result[i] = min_idx + 1  # 返回索引并加 1（使索引从 1 开始）

    #     return result

    #### 矩阵形式
    def ts_argmin(X):
        if len(X) <= 10:
            return np.ones_like(X)

        # Reshape the input data to 2D (nombre_ticker, nombre_date)
        X = X.reshape(nombre_ticker, nombre_date).T

        # Initialize the result array
        result = np.empty_like(X, dtype=np.float64)

        # Use sliding window view to create sliding windows of data
        windows = sliding_window_view(X, window, axis=0)

        # Apply argmin on each window (ignoring NaNs by converting them to inf)
        min_indices = np.argmin(np.nan_to_num(windows, nan=np.inf), axis=2)

        # Assign the result
        result[window-1:] = min_indices + 1  # Adding 1 to make the index 1-based

        # Handle the first `window-1` elements (initialize with the first valid value or zero)
        for j in range(X.shape[1]):
            first_non_nan = np.nan
            for i in range(window - 1, -1, -1):
                if not np.isnan(X[i, j]):
                    first_non_nan = X[i, j]
                    break
            if not np.isnan(first_non_nan):
                result[:window, j] = result[window-1, j]
            else:
                result[:window, j] = 0  # If all values are NaN, fill with 0
        del X, windows, min_indices
        return result.T.flatten()


    ts_argmin.__name__ = f"ts_argmin_{window}"
    return ts_argmin



dict_functions = {f"ts_argmin_{i}": all_function_18(i) for i in window_list}
ts_argmin_functions , ts_argmin_names = functions_name_arity1(dict_functions)



def all_function_19(window):
    # def ts_argmin(sr):
    #     n = len(sr)
    #     result = np.empty(n)  # 创建一个结果数组
    #     first_non_nan = sr[0]
    #     for i in range(n):
    #         if not np.isnan(sr[i]):
    #             first_non_nan = sr[i]
    #             break
    #     result[:window] = first_non_nan
    #     for i in range(window - 1, n):
    #         window_data = sr[i - window + 1:i + 1]  # 获取当前窗口的数据
    #         min_idx = np.argmin(window_data)  # 计算最小值的索引
    #         result[i] = min_idx + 1  # 返回索引并加 1（使索引从 1 开始）

    #     return result

    #### 矩阵形式
    def ts_argmin(X):
        if len(X) <= 10:
            return np.ones_like(X)

        # Reshape the input data to 2D (nombre_ticker, nombre_date)
        X = X.reshape(nombre_ticker, nombre_date).T

        # Initialize the result array
        result = np.empty_like(X, dtype=np.float64)

        # Use sliding window view to create sliding windows of data
        windows = sliding_window_view(X, window, axis=0)

        # Apply argmin on each window (ignoring NaNs by converting them to inf)
        min_indices = np.argmin(np.nan_to_num(windows, nan=np.inf), axis=2)

        # Assign the result
        result[window-1:] = min_indices + 1  # Adding 1 to make the index 1-based

        # Handle the first `window-1` elements (initialize with the first valid value or zero)
        for j in range(X.shape[1]):
            first_non_nan = np.nan
            for i in range(window - 1, -1, -1):
                if not np.isnan(X[i, j]):
                    first_non_nan = X[i, j]
                    break
            if not np.isnan(first_non_nan):
                result[:window, j] = result[window-1, j]
            else:
                result[:window, j] = 0  # If all values are NaN, fill with 0
        del X, windows, min_indices
        return result.T.flatten()


    ts_argmin.__name__ = f"ts_argmin_{window}"
    return ts_argmin


dict_functions = {f"ts_argmax_{i}": all_function_19(i) for i in window_list}
ts_argmax_functions , ts_argmax_names = functions_name_arity1(dict_functions)



def all_function_20(window):
    # 边界问题
    # def range_count(data):  # 滚动计算区间内值个数


    #     value = np.zeros(data.shape)
    #     value[:] = np.nan

    #     for i in range(window - 1, len(data)):
    #         x = data[i - window + 1:i + 1]

    #         value[i] = np.count_nonzero(((x >= (x.mean() - 1 * x.std())) & (x <= (x.mean() + 1 * x.std()))))

    #     value = np.where(np.isnan(value), 1e-10, value)
    #     value = np.where(np.isinf(value), 1e+30, value)

    #     return value

    #### 矩阵形式
    def range_count(data):
        if len(data) <= 10:
            return np.ones_like(data)

        # Reshape the input data to 2D (nombre_ticker, nombre_date)
        data = data.reshape(nombre_ticker, nombre_date).T

        # Initialize the result array
        result = np.empty_like(data, dtype=np.float64)

        # Use sliding window view to create sliding windows of data
        windows = sliding_window_view(data, window, axis=0)

        # Calculate the mean and standard deviation for each window
        means = np.nanmean(windows, axis=2)
        stds = np.nanstd(windows, axis=2)

        # Count values within the range [mean - std, mean + std] for each window
        counts = np.count_nonzero((windows >= (means[:, :, np.newaxis] - stds[:, :, np.newaxis])) &
                                  (windows <= (means[:, :, np.newaxis] + stds[:, :, np.newaxis])), axis=2)

        # Assign the counts to the result array
        result[window-1:] = counts

        # Handle the first `window-1` elements (initialize with NaN)
        result[:window-1,:] = result[window-1,:]

        # Replace NaN and Inf values
        result = np.where(np.isnan(result), 1e-10, result)
        result = np.where(np.isinf(result), 1e+30, result)
        del data, windows, means, stds, counts
        return result.T.flatten()
    range_count.__name__ = f"range_count_{window}"
    return range_count



dict_functions = {f"range_count_{i}": all_function_20(i) for i in window_list}
range_count_functions , range_count_names = functions_name_arity1(dict_functions)




def all_function_21(window):
    # 边界问题
    # def mean_diff(data):  # 滚动逐差的均值
    #     value = np.zeros(data.shape)
    #     value[:] = np.nan

    #     for i in range(window - 1, len(data)):
    #         diff_sum = 0.0
    #         for j in range(i - window + 1, i):
    #             diff_sum += data[j + 1] - data[j]
    #         value[i] = diff_sum / window

    #     value = np.where(np.isnan(value), 1e-10, value)
    #     value = np.where(np.isinf(value), 1e+30, value)

    #     return value

    #### 矩阵形式

    def mean_diff(data):
        if len(data) <= 10:
            return np.ones_like(data)

        # Reshape the input data to 2D (nombre_ticker, nombre_date)
        data = data.reshape(nombre_ticker, nombre_date).T

        # Initialize the result array
        result = np.empty_like(data, dtype=np.float64)

        # Use sliding window view to create sliding windows of data
        windows = sliding_window_view(data, window, axis=0)

        # Calculate the differences between consecutive values in each window
        diffs = np.diff(windows, axis=2)  # Diff between consecutive values along window axis

        # Use np.isnan to handle NaN values and count the valid differences
        valid_diffs = np.isfinite(diffs)  # Mask to check which diffs are valid (not NaN)
        valid_count = np.sum(valid_diffs, axis=2)  # Count of valid differences

        # Calculate the mean of the valid differences for each window
        mean_diffs = np.sum(diffs, axis=2) / valid_count  # Sum of diffs divided by count

        # Assign the result
        result[window-1:] = mean_diffs

        # Handle the first `window-1` elements (initialize with NaN or 0)

        result[:window-1,:] = result[window-1,:]

        # Replace NaN and Inf values
        result = np.where(np.isnan(result), 1e-10, result)
        result = np.where(np.isinf(result), 1e+30, result)
        del data, windows, diffs, valid_diffs, valid_count, mean_diffs
        return result.T.flatten()

    mean_diff.__name__ = f"mean_diff_{window}"
    return mean_diff



dict_functions = {f"mean_diff_{i}": all_function_21(i) for i in window_list}
mean_diff_functions , mean_diff_names = functions_name_arity1(dict_functions)


def all_function_22(window):
  # 边界问题
    # def rolling_linear_decay_mean(data):  # 滑动窗口的线性衰减平均值

    #     value = np.zeros(data.shape)
    #     value[:] = np.nan

    #     decay = np.linspace(1, 0, window)
    #     weights = decay / decay.sum()

    #     for i in range(window - 1, len(data)):
    #         value[i] = np.dot(data[i - window + 1:i + 1], weights)

    #     value = np.where(np.isnan(value), 1e-10, value)
    #     value = np.where(np.isinf(value), 1e+30, value)

    #     return value

    #### 矩阵形式
    def rolling_linear_decay_mean(data):
        if len(data) <= 10:
            return np.ones_like(data)

        # Reshape the input data to 2D (nombre_ticker, nombre_date)
        data = data.reshape(nombre_ticker, nombre_date).T

        # Initialize the result array
        result = np.empty_like(data, dtype=np.float64)

        # Create linear decay weights and normalize them
        decay = np.linspace(1, 0, window)  # Linear decay weights
        weights = decay / decay.sum()  # Normalize weights

        # Use sliding window view to create sliding windows of data
        windows = sliding_window_view(data, window, axis=0)

        # Apply the weighted mean calculation for each window
        weighted_means = np.sum(np.nan_to_num(windows, nan=0.0) * weights, axis=2)

        # Assign the weighted mean to the result
        result[window-1:] = weighted_means

        # Handle the first `window-1` elements (initialize with NaN or 0)
        result[:window-1,:] = result[window-1,:]

        # Replace NaN and Inf values
        result = np.where(np.isnan(result), 1e-10, result)
        result = np.where(np.isinf(result), 1e+30, result)
        del data,decay,weights,windows, weighted_means
        return result.T.flatten()


    rolling_linear_decay_mean.__name__ = f"rolling_linear_decay_mean_{window}"
    return rolling_linear_decay_mean

dict_functions = {f"rolling_linear_decay_mean_{i}": all_function_22(i) for i in window_list}
rolling_linear_decay_mean_functions , rolling_linear_decay_mean_names= functions_name_arity1(dict_functions)


@njit
def custom_diff(arr):
    return np.array([arr[i] - arr[i-1] for i in range(1, len(arr))])
def all_function_23(window):
    # 边界问题
    @njit
    # def ts_convexity(data):  # 滚动价格凸性
    #     rolling_price = np.zeros(data.shape)
    #     rolling_price[:] = np.nan

    #     convexity = np.zeros(data.shape)
    #     convexity[:] = np.nan

    #     for i in range(window - 1, len(data)):
    #         rolling_price[i] = np.mean(data[i - window + 1:i + 1])

    #     for i in range(window * 2 - 1, len(data)):
    #         changes = np.diff(
    #             np.log(rolling_price[i - window * 2 + 2:i + 1]))  # 计算对数收益率的一阶差分

    #         convexity[i] = np.sum(np.diff(changes))  # 计算二阶差分

    #     value = convexity
    #     value = np.where(np.isnan(value), 1e-10, value)
    #     value = np.where(np.isinf(value), 1e+30, value)

        # return value

    ### 矩阵形式
    def ts_convexity(data):
        if len(data) <= 10:
            return np.ones_like(data)

        rolling_price = np.empty(len(data), dtype=np.float64)  # Stores rolling price means
        convexity = np.empty(len(data), dtype=np.float64)  # Stores convexity values
        rolling_price[:] = np.nan
        convexity[:] = np.nan

        # Loop over the tickers (rows)
        for j in range(nombre_ticker):
            # Loop over the dates (columns) starting from the window
            for i in range(window - 1, nombre_date):
                # Get the linear index for data
                index_current = j * nombre_date + i
                index_window_start = j * nombre_date + (i - window + 1)

                # Get the window data
                window_data = data[index_window_start:index_current + 1]

                # Calculate rolling price mean
                rolling_price[index_current] = np.mean(window_data)

            # Calculate convexity
            for i in range(window * 2 - 1, nombre_date):
                index_current = j * nombre_date + i
                index_window_start = j * nombre_date + (i - window * 2 + 2)

                # Calculate log returns from rolling price mean
                log_returns = custom_diff(np.log(rolling_price[index_window_start:index_current + 1]))
                convexity[index_current] = np.sum(custom_diff(log_returns))

        # Replace NaN and Inf values
        convexity = np.where(np.isnan(convexity), 1e-10, convexity)
        convexity = np.where(np.isinf(convexity), 1e+30, convexity)

        return convexity


    ts_convexity.__name__ = f"ts_convexity_{window}"
    return ts_convexity


dict_functions = {f"ts_convexity_{i}": all_function_23(i) for i in window_list}
ts_convexity_functions , ts_convexity_names= functions_name_arity1(dict_functions)





def all_function_24(window):
      # 边界问题

    # def ts_scale(data):  # 滑动整体绝对值占比
    #     value = np.zeros(data.shape)
    #     value[:] = np.nan
    #     for i in range(window - 1, len(data)):
    #         tsscale = data[i] / (np.abs(data[i - window + 1:i + 1]).sum() + 1e-10)
    #         value[i] = tsscale

    #     value = np.where(np.isnan(value), 1e-10, value)
    #     value = np.where(np.isinf(value), 1e+30, value)

    #     return value

    #### 矩阵形式
    def ts_scale(data):

        if len(data) <= 10:
            return np.ones_like(data)

        # Reshape the input data to 2D (nombre_ticker, nombre_date)
        data = data.reshape(nombre_ticker, nombre_date).T

        # Initialize the result array
        result = np.empty_like(data, dtype=np.float64)

        # Use sliding window view to create sliding windows of data
        windows = sliding_window_view(data, window, axis=0)

        # Calculate the absolute sum for each window, ignoring NaNs
        abs_sum = np.nansum(np.abs(windows), axis=2) + 1e-10  # Adding small constant to avoid division by zero

        # Calculate the scaling factor for each window
        result[window-1:] = data[window-1:] / abs_sum

        # Handle the first `window-1` elements (initialize with NaN or 0)
        result[:window-1,:] = result[window-1,:]

        # Replace NaN and Inf values
        result = np.nan_to_num(result, nan=1e-10, posinf=1e+30, neginf=-1e+30)
        del data,windows, abs_sum
        return result.T.flatten()
    ts_scale.__name__ = f"ts_scale_{window}"
    return ts_scale

dict_functions = {f"ts_scale_{i}": all_function_24(i) for i in window_list}
ts_scale_functions , ts_scale_names = functions_name_arity1(dict_functions)


#### 第二组 series

In [27]:



def all_function_30(window):

# changed with 2d ema


    # def EMA(X, window):
    #     if len(X) <= 10:
    #       return np.ones_like(X)
    #     else:
    #         X, _l = handle_nan(X)
    #         X = X[_l:]
    #         if len(X) < window:
    #           return np.array([np.nan] * (len(X) + _l))
    #         kt = 2 / (window + 1)
    #         pre_ma = np.mean(X[:window])
    #         __res = np.array([np.nan] * (len(X) + _l))
    #         __res[_l + window - 1] = pre_ma
    #         for i in range(window, len(X)):
    #           pre_ma += (X[i] - pre_ma) * kt
    #           __res[_l + i] = pre_ma
    #         return __res
    def DEMA(X):
        def EMA(X, window):
            if len(X) <= 10:
                return np.ones_like(X)

            # Reshape the data to (nombre_ticker, nombre_date)
            X = X.reshape(nombre_ticker, nombre_date).T

            # Initialize result array
            result = np.full_like(X, np.nan, dtype=np.float64)

            # Calculate smoothing factor (alpha)
            alpha = 2 / (window + 1)

            # Compute the first EMA value using the mean of the first 'window' values
            result[window-1, :] = np.nanmean(X[:window, :], axis=0)

            # Use vectorized EMA calculation for the rest of the values
            for i in range(window, X.shape[0]):
                result[i, :] = alpha * X[i, :] + (1 - alpha) * result[i - 1, :]

            return result.T.flatten()

        if len(X) <= 10:
            return np.ones_like(X)

        # Compute the first and second EMA using the optimized EMA function
        ema1 = EMA(X, window)
        ema2 = EMA(ema1, window)

        # Calculate DEMA as 2 * EMA1 - EMA2
        result = 2 * ema1 - ema2
        for j in range(nombre_ticker):
            for i in range(window):

                    # Use the calculated value from the `window + 1` index
                    result[j * nombre_date + i] = result[j * nombre_date+window-1]
        del X,ema1,ema2
        return result
    DEMA.__name__ = f"DEMA_{window}"
    return DEMA




dict_functions = {f"DEMA_{i}": all_function_30(i) for i in window_list}
DEMA_functions , DEMA_names = functions_name_arity1(dict_functions)

def all_function_32(window):


    # def KAMA(X):
    #     if len(X) <= 10:
    #        return np.ones_like(X)
    #     else:
    #         X, _l = handle_nan(X)
    #         X = X[_l:]
    #         if len(X) < window:
    #             return np.array([np.nan] * (len(X) + _l))
    #         _af = 2 / (2 + 1)
    #         _as = 2 / (30 + 1)
    #         __res = np.array([np.nan] * (len(X) + _l))
    #         for i in range(window, len(X)):
    #             period_roc = X[i] - X[i - window]
    #             sum_roc = np.sum(np.abs(np.diff(X[i - window: i + 1])))
    #             _er = 1.0 if ((period_roc >= sum_roc) or (sum_roc == 0)) else abs(period_roc / sum_roc)
    #             _at = (_er * (_af - _as) + _as) ** 2
    #             __res[_l + i] = _at * X[i] + (1 - _at) * (__res[_l + i - 1] if i != window else X[i - 1])
    #         return __res
    # 矩阵形式
    def KAMA(X): ## 快但可能是错的
        if len(X) <= 10:
            return np.ones_like(X)

        # Reshape the input data to 2D (nombre_ticker, nombre_date)
        X = X.reshape(nombre_ticker, nombre_date).T

        # Initialize the result array
        __res = np.full_like(X, np.nan, dtype=np.float64)

        # KAMA parameters
        _af = 2 / (2 + 1)  # Fast smoothing constant
        _as = 2 / (30 + 1)  # Slow smoothing constant

        # Vectorized NaN handling: using np.nan_to_num to replace NaNs with zeros
        X = np.nan_to_num(X, nan=0.0)

        # Calculate the first KAMA value using the mean of the first 'window' data points
        __res[window - 1, :] = np.mean(X[:window, :], axis=0)

        # Use sliding_window_view to create sliding windows for the data
        windows = sliding_window_view(X, window, axis=0)

        # Calculate period ROC (Rate of Change) for each window (current price - price window periods ago)
        period_roc = windows[:, :, -1] - windows[:, :, 0]

        # Calculate the sum of absolute differences for each window (volatility)
        sum_roc = np.sum(np.abs(np.diff(windows, axis=2)), axis=2)

        # Calculate the Efficient Ratio (ER) as abs(period_roc) / sum_roc
        _er = np.abs(period_roc) / (sum_roc + 1e-10)

        # Calculate the smoothing constant (AT)
        _at = (_er * (_af - _as) + _as) ** 2

        # Clip _at to a reasonable range to avoid overflow
        _at = np.clip(_at, 1e-5, 1.0)

        # Compute KAMA recursively using the calculated smoothing constant
        for i in range(window, X.shape[0]):
            __res[i, :] = _at[i - window, :] * X[i, :] + (1 - _at[i - window, :]) * __res[i - 1, :]
        del X,_af,_as,windows, period_roc, sum_roc, _er, _at
        # Return the result after flattening and transposing
        __res[:window-1,:] = __res[window-1,:]

        return __res.T.flatten()

    KAMA.__name__ = f"KAMA_{window}"
    return KAMA

dict_functions = {f"KAMA_{i}": all_function_32(i) for i in window_list}
KAMA_functions , KAMA_names = functions_name_arity1(dict_functions)

def all_function_35(window):
  # 边界问题

    # def LINEARREG_SLOPE(X):
    #     if len(X) <= 10:
    #        return np.ones_like(X)
    #     else:
    #         Y = np.arange(window)
    #         res = np.full(len(X), np.nan)
    #         for i in range(window- 1, len(X)):
    #             X_slice = X[i -window + 1: i + 1]
    #             Y_slice = Y[:len(X_slice)]
    #             X_mean = np.mean(X_slice)
    #             Y_mean = np.mean(Y_slice)
    #             numerator = np.sum((X_slice - X_mean) * (Y_slice - Y_mean))
    #             denominator = np.sum((X_slice - X_mean) ** 2)
    #             denominator = denominator if denominator > 0.001 else 0.001
    #             res[i] = numerator / denominator
    #         return res
    # LINEARREG_SLOPE.__name__ = f"LINEARREG_SLOPE_{window}"
    # return LINEARREG_SLOPE

    ##### 矩阵形式
    def LINEARREG_SLOPE(X):
        if len(X) <= 10:
            return np.ones_like(X)

        # Reshape X to match the expected 2D structure
        # Reshape X to match the expected 2D structure
        X = X.reshape(nombre_ticker, nombre_date).T  # Reshape to [date, ticker]

        Y = np.arange(window)  # Time series for slope calculation

        # Create sliding window views for X and Y
        X_windows = sliding_window_view(X, window,axis=0)  # Shape: (num_dates - window + 1, window, num_tickers)
        Y_windows = np.tile(Y, (nombre_date - window + 1,nombre_ticker, 1))

        # Initialize result matrix
        res = np.full_like(X, np.nan)

        # Calculate means for X and Y


        X_mean = np.mean(X_windows, axis=2)  # Mean across windows for X
        Y_mean = np.mean(Y_windows, axis=2)  # Mean for Y

        # Calculate numerator and denominator for slope
        numerator = np.sum((X_windows - X_mean[:, :,np.newaxis]) * (Y_windows - Y_mean[:, :,np.newaxis]), axis=2)
        denominator = np.sum((Y_windows - Y_mean[:, :,np.newaxis]) ** 2, axis=2)

        # Avoid division by zero
        denominator = np.where(denominator > 0.001, denominator, 0.001)

        # Calculate the slope for each window and assign it to the result
        res[window - 1:] = numerator / denominator

        # Fill the first window-1 rows with the slope value of the first valid window
        res[:window - 1] = res[window - 1]
        del X,Y,X_windows,Y_windows,X_mean,Y_mean,numerator,denominator
        return res.T.flatten()



    LINEARREG_SLOPE.__name__ = f"LINEARREG_SLOPE_{window}"
    return LINEARREG_SLOPE

dict_functions = {f"LINEARREG_SLOPE_{i}": all_function_35(i) for i in window_list}
LINEARREG_SLOPE_functions , LINEARREG_SLOPE_names = functions_name_arity1(dict_functions)


def all_function_36(window):
    # def LINEARREG_ANGLE(X):
    #     if len(X) <= 10:
    #        return np.ones_like(X)
    #     else:
    #         Y = np.arange(window)
    #         res = np.full(len(X), np.nan)
    #         for i in range(window - 1, len(X)):
    #             X_slice = X[i - window + 1: i + 1]
    #             Y_slice = Y[:len(X_slice)]
    #             X_mean = np.mean(X_slice)
    #             Y_mean = np.mean(Y_slice)
    #             numerator = np.sum((X_slice - X_mean) * (Y_slice - Y_mean))
    #             denominator = np.sum((X_slice - X_mean) ** 2)
    #             denominator = denominator if denominator > 0.001 else 0.001
    #             res[i] = np.arctan(numerator / denominator) * (180.0 / np.pi)
    #         return res

    ##### 矩阵形式
    def LINEARREG_ANGLE(X):

        if len(X) <= 10:
            return np.ones_like(X)

        # Reshape X to match the expected 2D structure
        # Reshape X to match the expected 2D structure
        X = X.reshape(nombre_ticker, nombre_date).T  # Reshape to [date, ticker]

        Y = np.arange(window)  # Time series for slope calculation

        # Create sliding window views for X and Y
        X_windows = sliding_window_view(X, window,axis=0)  # Shape: (num_dates - window + 1, window, num_tickers)
        Y_windows = np.tile(Y, (nombre_date - window + 1,nombre_ticker, 1))

        # Initialize result matrix
        res = np.full_like(X, np.nan)

        # Calculate means for X and Y


        X_mean = np.mean(X_windows, axis=2)  # Mean across windows for X
        Y_mean = np.mean(Y_windows, axis=2)  # Mean for Y

        # Calculate numerator and denominator for slope
        numerator = np.sum((X_windows - X_mean[:, :,np.newaxis]) * (Y_windows - Y_mean[:, :,np.newaxis]), axis=2)
        denominator = np.sum((Y_windows - Y_mean[:, :,np.newaxis]) ** 2, axis=2)

        # Avoid division by zero
        denominator = np.where(denominator > 0.001, denominator, 0.001)

        # Calculate the slope for each window and assign it to the result
        res[window - 1:] = np.arctan(numerator / denominator) * (180.0 / np.pi)

        # Fill the first window-1 rows with the slope value of the first valid window
        res[:window - 1] = res[window - 1]
        del X,Y,X_windows,Y_windows,X_mean,Y_mean,numerator,denominator
        return res.T.flatten()

    LINEARREG_ANGLE.__name__ = f"LINEARREG_ANGLE_{window}"
    return LINEARREG_ANGLE

dict_functions = {f"LINEARREG_ANGLE_{i}": all_function_36(i) for i in window_list}
LINEARREG_ANGLE_functions , LINEARREG_ANGLE_names = functions_name_arity1(dict_functions)

def all_function_37(window):
    # def LINEARREG_INTERCEPT(X):
    #     if len(X) <= 10:
    #        return np.ones_like(X)
    #     else:
    #         Y = np.arange(window)
    #         res = np.full(len(X), np.nan)
    #         for i in range(window - 1, len(X)):
    #             X_slice = X[i - window + 1: i + 1]
    #             Y_slice = Y[:len(X_slice)]
    #             X_mean = np.mean(X_slice)
    #             Y_mean = np.mean(Y_slice)
    #             numerator = np.sum((X_slice - X_mean) * (Y_slice - Y_mean))
    #             denominator = np.sum((X_slice - X_mean) ** 2)
    #             denominator = denominator if denominator > 0.001 else 0.001
    #             _temp = np.arctan(numerator / denominator) * (180.0 / np.pi)
    #             res[i] = np.sum(X_slice) - _temp * np.sum(Y_slice)
    #         return res

    ##### 矩阵形式
    def LINEARREG_INTERCEPT(X):

        if len(X) <= 10:
            return np.ones_like(X)

        # Reshape X to match the expected 2D structure
        # Reshape X to match the expected 2D structure
        X = X.reshape(nombre_ticker, nombre_date).T  # Reshape to [date, ticker]

        Y = np.arange(window)  # Time series for slope calculation

        # Create sliding window views for X and Y
        X_windows = sliding_window_view(X, window,axis=0)  # Shape: (num_dates - window + 1, window, num_tickers)
        Y_windows = np.tile(Y, (nombre_date - window + 1,nombre_ticker, 1))

        # Initialize result matrix
        res = np.full_like(X, np.nan)

        # Calculate means for X and Y


        X_mean = np.mean(X_windows, axis=2)  # Mean across windows for X
        Y_mean = np.mean(Y_windows, axis=2)  # Mean for Y

        # Calculate numerator and denominator for slope
        numerator = np.sum((X_windows - X_mean[:, :,np.newaxis]) * (Y_windows - Y_mean[:, :,np.newaxis]), axis=2)
        denominator = np.sum((Y_windows - Y_mean[:, :,np.newaxis]) ** 2, axis=2)

        # Avoid division by zero
        denominator = np.where(denominator > 0.001, denominator, 0.001)

        # Calculate the slope for each window and assign it to the result
        res[window - 1:] = np.sum(X_windows,axis=2) - np.arctan(numerator / denominator) * (180.0 / np.pi) * np.sum(Y_windows,axis=2)

        # Fill the first window-1 rows with the slope value of the first valid window
        res[:window - 1] = res[window - 1]

        del X,Y,X_windows,Y_windows,X_mean,Y_mean,numerator,denominator
        return res.T.flatten()


    LINEARREG_INTERCEPT.__name__ = f"LINEARREG_INTERCEPT_{window}"
    return LINEARREG_INTERCEPT

dict_functions = {f"LINEARREG_INTERCEPT_{i}": all_function_37(i) for i in window_list}
LINEARREG_INTERCEPT_functions, LINEARREG_INTERCEPT_names = functions_name_arity1(dict_functions)




#### 第三组Section


In [28]:


def all_function_38(window):
# to consider

    def MAX_SECTION(X):
        if len(X) <= 10:
            return np.ones_like(X)

        # 重塑X为(nombres_ticker, nombres_date)的形状
        X = X.reshape(nombre_ticker, nombre_date).T

        # 使用 sliding_window_view 创建滑动窗口
        windows = sliding_window_view(X, window, axis=0)

        # 计算每个窗口的最大值
        max_values = np.max(windows, axis=2)

        # 创建结果数组
        result = np.empty_like(X, dtype=float)

        # 填充结果数组

        result[window-1:] = max_values
        result[:window] = result[window-1]

        del X,windows,max_values
        # 返回结果
        return result.T.flatten()

    MAX_SECTION.__name__ = f"MAX_SECTION_{window}"
    return MAX_SECTION

dict_functions = {f"MAX_SECTION_{i}": all_function_38(i) for i in window_list}
MAX_SECTION_functions , MAX_SECTION_names = functions_name_arity1(dict_functions)


def all_function_39(window):
# to consider

    def MIN_SECTION(X):
        if len(X) <= 10:
            return np.ones_like(X)

        # 重塑X为(nombres_ticker, nombres_date)的形状
        X = X.reshape(nombre_ticker, nombre_date).T

        # 使用 sliding_window_view 创建滑动窗口
        windows = sliding_window_view(X, window, axis=0)

        # 计算每个窗口的最大值
        min_values = np.min(windows, axis=2)

        # 创建结果数组
        result = np.empty_like(X, dtype=float)

        # 填充结果数组

        result[window-1:] = min_values
        result[:window] = result[window-1]

        del X,windows,min_values
        # 返回结果
        return result.T.flatten()
    MIN_SECTION.__name__ = f"MIN_SECTION_{window}"
    return MIN_SECTION


dict_functions = {f"MIN_SECTION_{i}": all_function_39(i) for i in window_list}
MIN_SECTION_functions, MIN_SECTION_names = functions_name_arity1(dict_functions)

def all_function_40(window):
# to consider

    def MEAN_SECTION(X):
        if len(X) <= 10:
            return np.ones_like(X)

        # 重塑X为(nombres_ticker, nombres_date)的形状
        X = X.reshape(nombre_ticker, nombre_date).T

        # 使用 sliding_window_view 创建滑动窗口
        windows = sliding_window_view(X, window, axis=0)

        # 计算每个窗口的最大值
        mean_values = np.mean(windows, axis=2)

        # 创建结果数组
        result = np.empty_like(X, dtype=float)

        # 填充结果数组

        result[window-1:] = mean_values
        result[:window] = result[window-1]

        del X,windows,mean_values
        # 返回结果
        return result.T.flatten()
    MEAN_SECTION.__name__ = f"MEAN_SECTION_{window}"
    return MEAN_SECTION

dict_functions = {f"MEAN_SECTION_{i}": all_function_40(i) for i in window_list}
MEAN_SECTION_functions , MEAN_SECTION_names = functions_name_arity1(dict_functions)



def all_function_41(window):
# to consider

    def MEDIAN_SECTION(X):
        if len(X) <= 10:
            return np.ones_like(X)

        # 重塑X为(nombres_ticker, nombres_date)的形状
        X = X.reshape(nombre_ticker, nombre_date).T

        # 使用 sliding_window_view 创建滑动窗口
        windows = sliding_window_view(X, window, axis=0)

        # 计算每个窗口的最大值
        median_values = np.median(windows, axis=2)

        # 创建结果数组
        result = np.empty_like(X, dtype=float)

        # 填充结果数组

        result[window-1:] = median_values
        result[:window] = result[window-1]
        del X,windows,median_values

        # 返回结果
        return result.T.flatten()
    MEDIAN_SECTION.__name__ = f"MEDIAN_SECTION_{window}"
    return MEDIAN_SECTION

dict_functions = {f"MEDIAN_SECTION_{i}": all_function_41(i) for i in window_list}
MEDIAN_SECTION_functions , MEDIAN_SECTION_names = functions_name_arity1(dict_functions)


def all_function_42(window):
# to consider

    def STD_SECTION(X):
        if len(X) <= 10:
            return np.ones_like(X)

        # 重塑X为(nombres_ticker, nombres_date)的形状
        X = X.reshape(nombre_ticker, nombre_date).T

        # 使用 sliding_window_view 创建滑动窗口
        windows = sliding_window_view(X, window, axis=0)

        # 计算每个窗口的最大值
        std_values = np.std(windows, axis=2)

        # 创建结果数组
        result = np.empty_like(X, dtype=float)

        # 填充结果数组

        result[window-1:] = std_values
        result[:window] = result[window-1]

        del X,windows,std_values
        # 返回结果
        return result.T.flatten()
    STD_SECTION.__name__ = f"STD_SECTION_{window}"
    return STD_SECTION

dict_functions = {f"STD_SECTION_{i}": all_function_42(i) for i in window_list}
STD_SECTION_functions , STD_SECTION_names = functions_name_arity1(dict_functions)

def all_function_43(window):
# to consider

    def NEUTRALIZE_SECTION(X):
        if len(X) <= 10:
            return np.ones_like(X)

        # 重塑X为(nombres_ticker, nombres_date)的形状
        X = X.reshape(nombre_ticker, nombre_date).T

        # 使用 sliding_window_view 创建滑动窗口
        windows = sliding_window_view(X, window, axis=0)

        # 计算每个窗口的最大值
        std_values = np.std(windows, axis=2)
        mean_values = np.mean(windows, axis=2)


        # 创建结果数组
        result = np.empty_like(X, dtype=float)

        # 填充结果数组

        result[window-1:] = (X[window-1:] - mean_values)/std_values
        result[:window] = result[window-1]

        del X,windows,std_values,mean_values
        # 返回结果
        return result.T.flatten()
    NEUTRALIZE_SECTION.__name__ = f"NEUTRALIZE_SECTION_{window}"
    return NEUTRALIZE_SECTION

dict_functions = {f"NEUTRALIZE_SECTION_{i}": all_function_43(i) for i in window_list}
NEUTRALIZE_SECTION_functions , NEUTRALIZE_SECTION_names = functions_name_arity1(dict_functions)

def all_function_44(window):
# to consider

    def RANK_SECTION(X):
        if len(X) <= 10:
            return np.ones_like(X)

        # 重塑X为(nombres_ticker, nombres_date)的形状
        X = X.reshape(nombre_ticker, nombre_date).T

        # 使用 sliding_window_view 创建滑动窗口
        windows = sliding_window_view(X, window, axis=0)

        # 计算每个窗口的最大值
        argmin_values = np.argmin(windows, axis=2)

        # 创建结果数组
        result = np.empty_like(X, dtype=float)

        # 填充结果数组

        result[window-1:] = argmin_values
        result[:window] = result[window-1]
        del X,windows,argmin_values

        # 返回结果
        return result.T.flatten()
    RANK_SECTION.__name__ = f"RANK_SECTION_{window}"
    return RANK_SECTION

dict_functions = {f"RANK_SECTION_{i}": all_function_44(i) for i in window_list}
RANK_SECTION_functions , RANK_SECTION_names = functions_name_arity1(dict_functions)


def all_function_45(window):
# to consider

    def SUM_SECTION(X):
        if len(X) <= 10:
            return np.ones_like(X)

        # 重塑X为(nombres_ticker, nombres_date)的形状
        X = X.reshape(nombre_ticker, nombre_date).T

        # 使用 sliding_window_view 创建滑动窗口
        windows = sliding_window_view(X, window, axis=0)

        # 计算每个窗口的最大值
        sum_values = np.sum(windows, axis=2)

        # 创建结果数组
        result = np.empty_like(X, dtype=float)

        # 填充结果数组

        result[window-1:] = sum_values
        result[:window] = result[window-1]

        del X,windows,sum_values
        # 返回结果
        return result.T.flatten()
    SUM_SECTION.__name__ = f"SUM_SECTION_{window}"
    return SUM_SECTION


dict_functions = {f"SUM_SECTION_{i}": all_function_45(i) for i in window_list}
SUM_SECTION_functions , SUM_SECTION_names = functions_name_arity1(dict_functions)


#### 第四组

In [29]:


def all_function_46(window):
  # checked
    # def ts_stddev2(X):
    #     if len(X) <= 10:
    #         return np.ones_like(X)
    #     X = X.reshape(nombre_ticker, nombre_date).T
    #     res =  np.empty_like(X)
    #     res[:] = np.nan
    #     for i in range(window - 1, len(X)):
    #         res[i] = np.nanstd(X[i - window + 1:i + 1])
    #     return res.T.flatten()
    # ts_stddev2.__name__ = f"ts_stddev2_{window}"
    # return ts_stddev2

    def ts_stddev2(X):
        """
        用向量化方法加速 ts_stddev2 的计算，并在函数结束时清理内存。
        """
        if len(X) <= 10:
            return np.ones_like(X)

        # 重构为二维数组，形状 (nombre_date, nombre_ticker)
        X2d = X.reshape(nombre_ticker, nombre_date).T.copy()
        n_dates, n_tickers = X2d.shape

        # 利用 sliding_window_view 构造滑动窗口
        X_windows = np.lib.stride_tricks.sliding_window_view(X2d, window, axis=0)

        # 计算均值和方差
        mean_windows = np.nanmean(X_windows, axis=2)
        diff = X_windows - mean_windows[..., np.newaxis]
        var_windows = np.nanmean(diff**2, axis=2)
        std_windows = np.sqrt(var_windows)

        # 构造结果矩阵
        result = np.full((n_dates, n_tickers), np.nan, dtype=np.float64)
        result[window - 1:, :] = std_windows
        result[:window-1,:] = result[window-1,:]

        # 在函数末尾删除临时变量，释放内存
        del X2d, X_windows, mean_windows, diff, var_windows, std_windows
        return result.T.flatten()
    ts_stddev2.__name__ = f"ts_stddev2_{window}"
    return ts_stddev2

dict_functions = {f"ts_stddev2_{i}": all_function_46(i) for i in window_list}
ts_stddev2_functions , ts_stddev2_names = functions_name_arity1(dict_functions)




def all_function_47(window):
# checked
    # def ts_corr2(X, Y):
    #     if len(X) <= 10:
    #         return np.ones_like(X)
    #     if len(Y) <= 10:
    #         return np.ones_like(Y)

    #     X = X.reshape(nombre_ticker, nombre_date).T
    #     Y = X.reshape(nombre_ticker, nombre_date).T

    #     res =  np.empty_like(X)
    #     min_len = min(len(X), len(Y))  # 确保 X 和 Y 长度一致
    #     for i in range(min_len - window + 1):
    #         X_ = X[i:i+window]
    #         Y_ = Y[i:i+window]
    #         valid_data_x = []
    #         valid_data_y = []
    #         # for idx in range(len(X_)):
    #         #     if not np.isnan(X_[idx]) and not np.isnan(Y_[idx]):
    #         #         valid_data_x.append(X_[idx])
    #         #         valid_data_y.append(Y_[idx])

    #         ##### 加速
    #         # Masking NaNs to get valid data
    #         valid_mask = ~np.isnan(X_) & ~np.isnan(Y_)
    #         valid_data_x = X_[valid_mask]
    #         valid_data_y = Y_[valid_mask]

    #         if len(valid_data_x) > 2:  # 确保有足够的数据点进行相关系数计算
    #             res[i+window-1] = np.corrcoef(valid_data_x, valid_data_y)[0, 1]
    #     return res.T.flatten()


    def ts_corr2(X, Y):
        """
        Calculates the ts_corr2 factor in 2D format.

        Args:
            X: First input data as a 1D numpy array.
            Y: Second input data as a 1D numpy array.
            window: Window size for calculation.
            nombre_ticker: Number of tickers.
            nombre_date: Number of dates.

        Returns:
            The ts_corr2 factor in 2D format as a numpy array.
        """
        if len(X) <= 10 or len(Y) <= 10:
            return np.ones_like(X)

        # Reshape X and Y to 2D
        X = X.reshape(nombre_ticker, nombre_date).T  # Reshape to [date, ticker]
        Y = Y.reshape(nombre_ticker, nombre_date).T  # Reshape to [date, ticker]

        # Create sliding windows for X and Y
        X_windows = sliding_window_view(X, window,0)
        Y_windows = sliding_window_view(Y, window,0)

        # Calculate means for window data
        X_mean = np.mean(X_windows, axis=2, keepdims=True)
        Y_mean = np.mean(Y_windows, axis=2, keepdims=True)

        # Calculate covariance (numerator)
        numerator = np.sum((X_windows - X_mean) * (Y_windows - Y_mean), axis=2)

        # Calculate variances (denominators)
        X_variance = np.sum((X_windows - X_mean) ** 2, axis=2)
        Y_variance = np.sum((Y_windows - Y_mean) ** 2, axis=2)

        # Calculate correlation using covariance / sqrt(variance_x * variance_y)
        denominator = np.sqrt(X_variance * Y_variance)
        denominator = np.where(denominator > 1e-10, denominator, 1e-10)  # Avoid division by zero

        # Calculate correlation
        correlation = numerator / denominator

        # Initialize result matrix
        result = np.full_like(X, np.nan, dtype=np.float64)
        result[window - 1:, :] = correlation  # Populate the result after the first full window
        result[:window-1,:] = result[window-1,:]
        del X,Y,X_windows, Y_windows, X_mean, Y_mean, numerator, X_variance, Y_variance, denominator, correlation
        return result.T.flatten()  # Flatten back to 1D

    ts_corr2.__name__ = f"ts_corr2_{window}"
    return ts_corr2


dict_functions = {f"ts_corr2_{i}": all_function_47(i) for i in window_list}
ts_corr2_functions , ts_corr2_names = functions_name_arity2(dict_functions)


def all_function_48(window):
    # @nb.njit
    def ts_freq2(X):
        if len(X) <= 10:
            return np.ones_like(X)
        # Reshape the input data to 2D (dates, tickers)
        X = X.reshape(nombre_ticker, nombre_date).T

        # Use sliding_window_view to create sliding windows for the data
        windows = sliding_window_view(X, window, axis=0)  # Shape: [num_dates - window + 1, window, num_tickers]
        # print(time.time() - s)
        # Peak detection in each window:
        # Compare each element in the window to its neighbors
        peaks = (windows[:, :,1:-1] > windows[:,  :,:-2,]) & (windows[:,  :,1:-1] > windows[:,  :,2:])
        # print(time.time() - s)
        # Count the number of peaks in each window
        peak_count = np.count_nonzero(peaks, axis=2)  # Shape: [num_dates - window + 1, num_tickers]
        # print(time.time() - s)
        # Initialize the result array with NaNs
        result = np.full_like(X, np.nan, dtype=np.float64)

        # Populate the result with the computed peak counts
        result[window - 1:, :] = peak_count
        result[:window, :] = result[window-1, :]
        del X,windows, peaks, peak_count
        return result.T.flatten()  # Flatten back to 1D

    ts_freq2.__name__ = f"ts_freq2_{window}"
    return ts_freq2

dict_functions = {f"ts_freq2_{i}": all_function_48(i) for i in window_list}
ts_freq2_functions , ts_freq2_names = functions_name_arity1(dict_functions)

def all_function_49(window):
    # 不理解 _l 目的
    def EMA2(X):
        if len(X) <= 10:
            return np.ones_like(X)

        # Reshape the data to (nombre_ticker, nombre_date)
        X = X.reshape(nombre_ticker, nombre_date).T

        # Initialize result array
        result = np.full_like(X, np.nan, dtype=np.float64)

        # Calculate smoothing factor (alpha)
        alpha = 2 / (window + 1)

        # Compute the first EMA value using the mean of the first 'window' values
        result[window-1, :] = np.nanmean(X[:window, :], axis=0)

        # Use vectorized EMA calculation for the rest of the values
        for i in range(window, X.shape[0]):
            result[i, :] = alpha * X[i, :] + (1 - alpha) * result[i - 1, :]
        result[:window-1,:] = result[window-1,:]

        del X, alpha
        return result.T.flatten()
    EMA2.__name__ = f"EMA2_{window}"
    return EMA2



dict_functions = {f"EMA2_{i}": all_function_49(i) for i in window_list}
EMA2_functions , EMA2_names = functions_name_arity1(dict_functions)




def all_function_51(window):
  # 不理解 _l 目的

    def MA2(X):
        if len(X) <= 10:
            return np.ones_like(X)

        # Reshape X into 2D (dates, tickers)
        X = X.reshape(nombre_ticker, nombre_date).T

        # Handle NaN values by replacing NaNs with the column means (or other strategies)
        # Optionally, use np.nan_to_num() if you want to replace NaNs with zeros or some other value
        X = np.nan_to_num(X, nan=np.nan)

        # Use sliding_window_view to create sliding windows for each column (ticker)
        windows = sliding_window_view(X, window, axis=0)

        # Calculate the mean for each window along the axis=1 (which is now the window axis)
        column_means = np.nanmean(windows, axis=2)  # This will compute the mean for each window

        # Initialize result array with NaN for positions where there isn't enough data to compute
        result = np.full_like(X, np.nan, dtype=np.float64)

        # Assign the calculated moving averages to the appropriate positions in the result array
        result[window - 1:, :] = column_means
        result[:window-1,:] = result[window-1,:]
        del X,windows, column_means
        return result.T.flatten()


    MA2.__name__ = f"MA2_{window}"
    return MA2

dict_functions = {f"MA2_{i}": all_function_51(i) for i in window_list}
MA2_functions , MA2_names = functions_name_arity1(dict_functions)


def all_function_53(window):
  # checked

    def MIDPOINT2(X):
        if len(X) <= 10:
            return np.ones_like(X)  # You can change this to np.zeros_like(X) or np.nan if preferred

        # Reshape the input data to 2D (dates, tickers)
        X = X.reshape(nombre_ticker, nombre_date).T

        # Use sliding_window_view to create sliding windows for the data
        windows = sliding_window_view(X, window, axis=0)  # Shape: [num_dates - window + 1, window, num_tickers]

        # Calculate the midpoint as (max + min) / 2 for each window
        max_vals = np.nanmax(windows, axis=2)  # Max for each window
        min_vals = np.nanmin(windows, axis=2)  # Min for each window

        # Calculate the midpoint for each window
        result = (max_vals + min_vals) / 2

        # Initialize result array and fill the first few NaN values
        final_result = np.full_like(X, np.nan, dtype=np.float64)
        final_result[window - 1:, :] = result  # Populate the result after the first full window

        final_result[:window-1,:] = result[window-1,:]

        del X,windows, max_vals, min_vals, result
        return final_result.T.flatten()  # Flatten back to 1D

    MIDPOINT2.__name__ = f"MIDPOINT2_{window}"
    return MIDPOINT2

dict_functions = {f"MIDPOINT2_{i}": all_function_53(i) for i in window_list}
MIDPOINT2_functions , MIDPOINT2_names = functions_name_arity1(dict_functions)



def all_function_54(window):
  # checked

    # def BETA2(X, Y):
    #     if len(X) <= 10:
    #        return np.ones_like(X)

    #     X = X.reshape(nombre_ticker, nombre_date).T
    #     Y = Y.reshape(nombre_ticker, nombre_date).T
    #     res =  np.empty_like(X)
    #     for i in range(window - 1, len(X)):
    #         X_slice = X[i - window + 1: i + 1]
    #         Y_slice = Y[i - window + 1: i + 1]
    #         X_mean = np.mean(X_slice)
    #         Y_mean = np.mean(Y_slice)
    #         numerator = np.sum((X_slice - X_mean) * (Y_slice - Y_mean))
    #         denominator = np.sum((X_slice - X_mean) ** 2)
    #         denominator = denominator if denominator > 0.001 else 0.001
    #         res[i] = numerator / denominator
    #     return res.T.flatten()


    def BETA2(X, Y):
        if len(X) <= 10 or len(Y) <= 10:
            return np.ones_like(X)

        # Reshape to 2D (dates, tickers)
        X = X.reshape(nombre_ticker, nombre_date).T
        Y = Y.reshape(nombre_ticker, nombre_date).T

        # Create sliding windows for X and Y
        X_windows = sliding_window_view(X, window, axis=0)  # Shape: [num_dates - window + 1, window, num_tickers]
        Y_windows = sliding_window_view(Y, window, axis=0)  # Shape: [num_dates - window + 1, window, num_tickers]

        # Initialize the result array with NaN values
        result = np.full_like(X, np.nan, dtype=np.float64)

        # Calculate the mean for each window (ignoring NaNs)
        X_means = np.nanmean(X_windows, axis=2)
        Y_means = np.nanmean(Y_windows, axis=2)

        # Calculate Beta for each window using vectorized operations
        numerators = np.nansum((X_windows - X_means[:, :, np.newaxis]) * (Y_windows - Y_means[:, :, np.newaxis]), axis=2)
        denominators = np.nansum((X_windows - X_means[:, :, np.newaxis]) ** 2, axis=2)

        # Avoid division by zero by replacing small denominators with a minimum value
        denominators = np.where(denominators > 1e-17, denominators, 1e-17)

        # Calculate Beta for each window
        result[window - 1:, :] = numerators / denominators
        result[:window-1,:] = result[window-1,:]

        del X,X_windows, Y_windows, X_means, Y_means, numerators, denominators
        return result.T.flatten()  # Flatten back to 1D


    BETA2.__name__ = f"BETA2_{window}"
    return BETA2


dict_functions = {f"BETA2_{i}": all_function_54(i) for i in window_list}
BETA2_functions , BETA2_names = functions_name_arity2(dict_functions)


## 还是jit快


@njit
def calc_skew_recurrence(sum_x, sum_x2, sum_x3, window):
    if window < 2:
        return np.nan
    mu = sum_x / window
    m2 = (sum_x2 / window) - (mu ** 2)
    if m2 == 0:
        return 0.0
    m3 = (sum_x3 / window) - 3 * mu * (sum_x2 / window) + 2 * (mu ** 3)
    skew = (window * m3) / ((window - 1) * (window - 2) * (m2 ** 1.5))
    return skew

def all_function_55(window):
# checked
    @njit
    def ts_skew(X):
        if len(X) <= 10:
            return np.ones_like(X)

        # 初始化结果数组
        res = np.full(X.shape, np.nan, dtype=np.float64)

        # 遍历每列（每个序列）
        for col in range(nombre_ticker):  # 并行遍历每个 ticker
            column_data = X[col * nombre_date : (col + 1) * nombre_date]

            # 预计算累积和、累积平方和、累积立方和
            cumsum = np.cumsum(column_data)
            cumsum_sq = np.cumsum(column_data ** 2)
            cumsum_cu = np.cumsum(column_data ** 3)

            # 滑动窗口计算
            for i in range(window - 1, nombre_date):
                start = i - window + 1
                # 使用线性索引来访问数据
                sum_x = cumsum[i] - (cumsum[start - 1] if start > 0 else 0)
                sum_x2 = cumsum_sq[i] - (cumsum_sq[start - 1] if start > 0 else 0)
                sum_x3 = cumsum_cu[i] - (cumsum_cu[start - 1] if start > 0 else 0)
                # 计算偏度
                res_index = col * nombre_date + i  # 计算线性索引
                res[res_index] = calc_skew_recurrence(sum_x, sum_x2, sum_x3, window)

        for j in range(nombre_ticker):
            for i in range(window):

                    # Use the calculated value from the `window + 1` index
                    res[j * nombre_date + i] = res[j * nombre_date+window-1]

        return res

    ts_skew.__name__ = f"ts_skew_{window}"
    return ts_skew


dict_functions = {f"ts_skew_{i}": all_function_55(i) for i in window_list}
ts_skew_functions , ts_skew_names = functions_name_arity1(dict_functions)


def all_function_56(window):
# checked
  def ts_autocorr(X):
      """
      Moving autocorrelation coefficient for each column of a matrix.

      Parameters:
          X (numpy.ndarray): Input matrix of shape (n_samples, n_features).
          d (int): Window size for moving calculation.
          lag (int): Lag period for autocorrelation.

      Returns:
          numpy.ndarray: Matrix of autocorrelation coefficients.
      """
      lag = 1
      if len(X) <= 10:
          return np.ones_like(X)

      # Reshape X to match the expected 2D structure [date, ticker]
      X = X.reshape(nombre_ticker, nombre_date).T

      # Create sliding windows for X and lagged X
      X_windows = sliding_window_view(X,window, axis=0)

      # Create lagged version of X by shifting the data by 'lag' periods
      lagged_X = np.roll(X, shift=lag, axis=0)
      lagged_X_windows = sliding_window_view(lagged_X, window,axis=0)

      lagged_X_mean = np.nanmean(lagged_X_windows, axis=2,keepdims=True)

      X_mean = np.nanmean(X_windows, axis=2,keepdims=True)

      # Calculate means for window data and lagged data across all windows
      covariance = np.sum((X_windows - X_mean) * (lagged_X_windows - lagged_X_mean), axis=2)
      variance_X = np.sum((X_windows - X_mean) ** 2, axis=2)
      variance_lagged_X = np.sum((lagged_X_windows - lagged_X_mean) ** 2, axis=2)

      # Calculate Pearson correlation coefficient (normalized covariance)
      correlation = covariance / np.sqrt(variance_X * variance_lagged_X)

      # Avoid division by zero
      correlation = np.where(np.isnan(correlation), 0, correlation)

      # Initialize result matrix
      result = np.full_like(X, np.nan, dtype=np.float64)
      result[window - 1:, :] = correlation  # Populate the result after the first full window
      result[:window, :] = result[window-1, :]

      del X,lagged_X, X_windows, lagged_X_windows,lagged_X_mean,X_mean, covariance, variance_X, variance_lagged_X, correlation
      return result.T.flatten()

  ts_autocorr.__name__ = f"ts_autocorr_{window}"
  return ts_autocorr


dict_functions = {f"ts_autocorr_{i}": all_function_56(i) for i in window_list}
ts_autocorr_functions , ts_autocorr_names = functions_name_arity1(dict_functions)



#### make function

In [30]:

#双变量
add_function          = make_function(function=add,        name='add',          arity=2)
sub_function          = make_function(function=sub,        name='sub',          arity=2)
mul_function          = make_function(function=mul,        name='mul',          arity=2)
div_function          = make_function(function=div,        name='div',          arity=2)
max_function          = make_function(function=max,        name='max',          arity=2)
min_function          = make_function(function=min,        name='min',          arity=2)
andpn_function        = make_function(function=_andpn,     name='andpn',        arity=2)
orpn_function         = make_function(function=_orpn,      name='orpn',         arity=2)
ltpn_function         = make_function(function=_ltpn,      name='ltpn',         arity=2)
gtpn_function          = make_function(function=_gtpn,     name='gtpn',         arity=2)
andp_function         = make_function(function=_andp,      name='andp',         arity=2)
orp_function          = make_function(function=_orp,       name='orp',          arity=2)
ltp_function          = make_function(function=_ltp,       name='ltp',          arity=2)
gtp_funtion           = make_function(function=_gtp,       name='gtp',          arity=2)
andn_function         = make_function(function=_andn,      name='andn',         arity=2)
orn_function          = make_function(function=_orn,       name='orn',          arity=2)
ltn_function          = make_function(function=_ltn,       name='ltn',          arity=2)
gtn_function          = make_function(function=_gtn,       name='gtn',          arity=2)
if_function           = make_function(function=_if,        name='if',           arity=3)
clear_by_cond_function= make_function(function=_clear_by_cond, name='clear_by_cond', arity=3)
if_then_else_function = make_function(function=_if_then_else, name='if_then_else', arity=3)
if_cond_then_else_function= make_function(function=_if_cond_then_else, name='if_cond_then_else', arity=4)
#单变量
cumulative_rank_function =make_function(function=cumulative_rank,       name='cumulative_rank',         arity=1)
sign_function         = make_function(function=sign,       name='sign',         arity=1)
sigmoid_function      = make_function(function=sigmoid,    name='sigmoid',      arity=1)
squre_function        = make_function(function=squre,      name='square',       arity=1)
sqrt_function         = make_function(function=sqrt,       name='sqrt',         arity=1)
log_function          = make_function(function=log,        name='log',          arity=1)
inv_function          = make_function(function=inv,        name='inv',          arity=1)
sin_function          = make_function(function=sin,        name='sin',          arity=1)
cos_function          = make_function(function=cos,        name='cos',          arity=1)
tan_function          = make_function(function=tan,        name='tan',          arity=1)
abs_function          = make_function(function=abs,        name='abs',          arity=1)
neg_function          = make_function(function=neg,        name='neg',          arity=1)
sum_function          = make_function(function=sum,        name='sum',          arity=1)
scale_functions       = make_function(function=scale,      name='scale',        arity=1)
delayy_function       = make_function(function=_delayy,    name='delayy',       arity=1)
delta_function        = make_function(function=_delta,     name='_delta',       arity=1)
signedpower_function  = make_function(function=_signedpower,name='_signedpower',arity=1)
stdd_function         = make_function(function=_stdd,      name='stdd',         arity=1)
rankk_function        = make_function(function=_rankk,     name='rankk',        arity=1)
combine_function      = make_function(function=_combine,   name='combine',      arity=2)
Rank_original_function= make_function(function=Rank_original,name='Rank_original',arity=1)
# Rank_columns_function = make_function(function=Rank_columns,name='Rank_columns',arity=1)
Rank_rows_function    = make_function(function=Rank_rows,   name='Rank_rows'   ,arity=1)
#横向
# normalize_ranks_function         = make_function(function=normalize_ranks,       name='normalize_ranks',         arity=1)
# FREQ_SECTION_function = make_function(function=FREQ_SECTION,name='FREQ_SECTION',arity=1)
z_score_function      = make_function(function=z_score,     name='z_score',     arity=1)
                #双变量


In [31]:
sub_func_set = [ andpn_function,orpn_function,ltpn_function,\
                gtpn_function,andp_function,orp_function,ltp_function,gtp_funtion,andn_function,orn_function,ltn_function,gtn_function,if_function,\
                clear_by_cond_function,if_then_else_function,\
                cumulative_rank_function,\
                #单变量
                sign_function,sigmoid_function, neg_function,\
                tan_function,sum_function, scale_functions,delayy_function, delta_function,signedpower_function, stdd_function,rankk_function,combine_function,\
                Rank_original_function,\
                # Rank_columns_function
                Rank_rows_function,\
                #横向
                z_score_function,if_cond_then_else_function\
                #第一组
                ]

In [32]:
basic_func_set = [
    add_function, sub_function, mul_function, div_function, squre_function, sqrt_function,
]

In [33]:
basic_func_set_potential = [
    max_function, min_function, log_function, inv_function, sin_function, cos_function, tan_function, abs_function
]

In [34]:
basic_func_sets = []
for i in range(100):
  basic_func_sets.extend(basic_func_set)

In [35]:
function_set = basic_func_sets + sub_func_set + basic_func_set_potential + delay_functions  + Delta2v_functions      + Cov_functions     + Prod_functions  + Tsrank_col_functions + Lowday_functions + Highday_functions + Decaylinear_functions + Wma_functions   +  Count_functions + Regbeta_functions + ts_argmin_functions\
                 + ts_argmax_functions + range_count_functions + mean_diff_functions + rolling_linear_decay_mean_functions + ts_convexity_functions + ts_scale_functions   + ts_corr2_functions   + ts_freq2_functions  + EMA2_functions  + DEMA_functions  + MA2_functions  \
                 + KAMA_functions + MIDPOINT2_functions + BETA2_functions + LINEARREG_SLOPE_functions + LINEARREG_ANGLE_functions + LINEARREG_INTERCEPT_functions + MAX_SECTION_functions +SUM_SECTION_functions + MIN_SECTION_functions + MEAN_SECTION_functions + MEDIAN_SECTION_functions+ STD_SECTION_functions \
                 + RANK_SECTION_functions +NEUTRALIZE_SECTION_functions  +ts_stddev2_functions
                 #+RETURN_functions+ts_skew_functions
                #  ts_stddev2_functions  +BETA2_functions
                #  + ts_corr2_functions +ts_freq2_functions +EMA2_functions  +MIDPOINT2_functions +ts_corr2_functions

In [36]:
num_columns = x1_np.shape[1]
print("Number of columns in x1_np:", num_columns)

Number of columns in x1_np: 5


In [37]:
factor=data_clean.drop('y',axis=1)

In [38]:
base_Factor =factor.columns.tolist()
base_Factor

['open_composed',
 'high_composed',
 'low_composed',
 'close_composed',
 'adjClose_composed']

In [39]:



# base_Factor0 = []
# for b in base_Factor:
#   if b == 'y':
#     pass
#   else:
#     base_Factor0.append(b)
# base_Factor = base_Factor0
# # base_Factor = selected_features

X_train_df = data_clean.drop('y',axis=1)
y_train_df = data_clean['y']

x=X_train_df.to_numpy(dtype=np.float64)
y=y_train_df.to_numpy(dtype=np.float64)

x1_df = X_train_df[X_train_df.index.get_level_values(1) <  '2021-01-01']
x1_np=x1_df.to_numpy(dtype=np.float64)


x2_df = X_train_df[X_train_df.index.get_level_values(1) >= '2021-01-01']
x2_np=x2_df.to_numpy(dtype=np.float64)

y1_df = y_train_df[y_train_df.index.get_level_values(1) <  '2021-01-01']
y1_np = y1_df.to_numpy(dtype=np.float64)

y2_df=y_train_df[y_train_df.index.get_level_values(1)   >= '2021-01-01']
y2_np=y2_df.to_numpy(dtype=np.float64)

In [40]:


start_time = time.time()

est_gp = SymbolicTransformer(feature_names    = base_Factor,         # 因子特征
                            function_set      = function_set,        # 算子式
                            # stopping_criteria = 1e-6,
                            metric            = rankic_normal,                           #my_metric_group,my_metric_ud,'spearman'
                            generations       = 10,
                            #  population_size= 400000 ,
                            population_size   = 20000 ,
                            tournament_size   = 50,      # 选择开始 重要的 算子的程度 ，越大，程度越大
                            init_depth        = (2,8),
                            random_state      = np.random.randint(10000),
                            n_components      = 100,
                            const_range       = (-1,1),
                            p_crossover       = 0.4,
                            p_subtree_mutation= 0.01,
                            p_hoist_mutation  = 0,
                            p_point_mutation  = 0.01,
                            p_point_replace = 0.4,
                            n_jobs=-1,
                            verbose=1,
                            low_memory=False)

# 模型拟合
est_gp.fit(x1_np, y1_np)

# 找出最优
best_programs = est_gp._best_programs
best_programs_dict = {}

# 记录结果
# 记录结果
for p in best_programs:
    factor_name = 'alpha_' + str(best_programs.index(p) + 1)
    best_programs_dict[factor_name] = {'fitness':p.fitness_, 'expression':str(p), 'depth':p.depth_, 'length':p.length_}

best_programs_dict = pd.DataFrame(best_programs_dict).T
best_programs_dict = best_programs_dict[best_programs_dict.fitness.abs() >= 0.0002]
best_programs_dict_positif = best_programs_dict.sort_values(by= 'fitness',ascending=True)
best_programs_dict_negatif = best_programs_dict.sort_values(by= 'fitness',ascending=False)
end_time = time.time()


total_time = (end_time - start_time)/3600
print(f"finished_time: {total_time} hours")

best_programs_dict_negatif.to_csv('/content/drive/MyDrive/Colab/table_result_without_jit_52.csv')


    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0     9.27      -0.00300655        6        0.0431456              N/A    959.61m


/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


   1     4.48        0.0121529        6        0.0475668              N/A    449.59m
   2     4.32        0.0143905        6        0.0498848              N/A    462.12m
   3     4.24        0.0326012        5        0.0514729              N/A    334.83m
   4     5.34        0.0381704        7        0.0524589              N/A    207.50m
   5     5.06        0.0378635        8        0.0525111              N/A    175.51m
   6     4.12        0.0357887        5        0.0523386              N/A    144.10m
   7     4.10        0.0359201        5        0.0523386              N/A     93.96m
   8     4.78        0.0374165        5        0.0523386              N/A     44.50m
   9     5.00        0.0392119        5        0.0523386              N/A      0.00s
finished_time: 9.105488720734915 hours


In [41]:
best_programs_dict_negatif

,fitness,expression,depth,length
alpha_1,0.047339,MAX_SECTION_70(MA2_149(ts_corr2_15(close_compo...,3,5
alpha_64,0.047339,MAX_SECTION_70(MA2_149(ts_corr2_15(close_compo...,3,5
alpha_74,0.047339,MAX_SECTION_70(MA2_149(ts_corr2_15(close_compo...,3,5
alpha_73,0.047339,MAX_SECTION_70(MA2_149(ts_corr2_15(close_compo...,3,5
alpha_72,0.047339,MAX_SECTION_70(MA2_149(ts_corr2_15(close_compo...,3,5
...,...,...,...,...
alpha_31,0.047339,MAX_SECTION_70(MA2_149(ts_corr2_15(close_compo...,3,5
alpha_30,0.047339,MAX_SECTION_70(MA2_149(ts_corr2_15(close_compo...,3,5
alpha_29,0.047339,MAX_SECTION_70(MA2_149(ts_corr2_15(close_compo...,3,5
alpha_28,0.047339,MAX_SECTION_70(MA2_149(ts_corr2_15(close_compo...,3,5


In [42]:
best_programs[10].fitness_

0.047338553286960565

#Train

##读取数据

In [86]:
file_path_1 = "/content/drive/MyDrive/Colab/data_clean_合集_最新.pkl"
data_clean=pd.read_pickle(file_path_1)
base_Factor = data_clean.columns.tolist()
data_clean=pd.read_pickle(file_path_1)
# data_clean.fillna(method='ffill', inplace=True)  # 向前填充
# data_clean.fillna(method='bfill', inplace=True)
data = data_clean
# data.replace(0, np.nan, inplace=True)
# data.fillna(method='ffill', inplace=True)
# data.fillna(method='bfill', inplace=True)
data

open      high       low     close  adjClose      vwap  \
ticker date                                                                     
0050   2010-01-05 -0.687952 -0.688420 -0.688152 -0.688480 -0.686910 -0.688534   
       2010-01-06 -0.688103 -0.687672 -0.687792 -0.687571 -0.686153 -0.688067   
       2010-01-07 -0.687267 -0.687388 -0.687188 -0.687717 -0.686275 -0.687673   
       2010-01-08 -0.687461 -0.687658 -0.687343 -0.687399 -0.686010 -0.687748   
       2010-01-11 -0.687519 -0.687323 -0.687204 -0.687181 -0.685828 -0.687589   
...                     ...       ...       ...       ...       ...       ...   
XSMI   2024-06-24  2.758994  2.784666  2.780434  2.802151  3.380982  2.782671   
       2024-06-25  2.785562  2.766568  2.792536  2.773880  3.348156  2.780705   
       2024-06-26  2.781439  2.766902  2.790117  2.770354  3.344061  2.778274   
       2024-06-27  2.751408  2.737581  2.762153  2.757594  3.329243  2.753243   
       2024-06-28  2.747871  2.730737  2.750774  2.733459  3.301216  2.741772   

                   open_composed  high_composed  low_composed  close_composed  \
ticker date                                                                     
0050   2010-01-05      -0.258004      -0.263814     -0.257305       -0.260749   
       2010-01-06      -0.258044      -0.263636     -0.257056       -0.260265   
       2010-01-07      -0.257671      -0.263533     -0.256978       -0.260577   
       2010-01-08      -0.257922      -0.263867     -0.257266       -0.260663   
       2010-01-11      -0.258031      -0.263707     -0.257002       -0.260414   
...                          ...            ...           ...             ...   
XSMI   2024-06-24       2.339524       2.359556      2.361499        2.374765   
       2024-06-25       2.362142       2.355507      2.378049        2.361970   
       2024-06-26       2.374585       2.359620      2.369213        2.349027   
       2024-06-27       2.361202       2.341013      2.369747        2.350958   
       2024-06-28       2.365084       2.342640      2.347145        2.326038   

                   adjClose_composed  vwap_composed   returns  \
ticker date                                                     
0050   2010-01-05          -0.265271      -0.259975 -0.590986   
       2010-01-06          -0.265076      -0.259758  1.477777   
       2010-01-07          -0.264911      -0.259697 -0.274129   
       2010-01-08          -0.265171      -0.259936  0.484504   
       2010-01-11          -0.265050      -0.259796  0.319309   
...                              ...            ...       ...   
XSMI   2024-06-24           2.676255       2.356167  0.855162   
       2024-06-25           2.661981       2.361688 -0.625792   
       2024-06-26           2.647542       2.360411 -0.109753   
       2024-06-27           2.649696       2.352996 -0.304380   
       2024-06-28           2.621897       2.342557 -0.545867   

                   returns_composed  CDL2CROWS_cumsum  CDL3OUTSIDE_cumsum  \
ticker date                                                                 
0050   2010-01-05         -0.619335          0.237521            0.236719   
       2010-01-06          1.140136          0.237521            0.236719   
       2010-01-07         -0.795143          0.237521            0.236719   
       2010-01-08         -0.251136          0.237521            0.236719   
       2010-01-11          0.562226          0.237521            0.237216   
...                             ...               ...                 ...   
XSMI   2024-06-24          0.681338         -0.799075           -0.715569   
       2024-06-25         -0.405504         -0.799075           -0.715569   
       2024-06-26         -0.411490         -0.799075           -0.715569   
       2024-06-27          0.012047         -0.799075           -0.715569   
       2024-06-28         -0.755222         -0.799075           -0.715569   

                   CDLSHOOTINGSTAR_composed_cumsum  CDLENGULFING_cu

In [87]:



# file_path_2='/content/drive/MyDrive/Colab/full_etf_with_ind_no_fill.pkl'
# data=pd.read_pickle(file_path_2)
# price=data[['close']]
# data_clean['y'] = price['close'].unstack('ticker').pct_change(5).shift(-6).unstack()
# data_clean.replace([0.0,0], np.nan, inplace=True)
# data_clean.interpolate(method='linear', inplace=True)




X_train_df = data[base_Factor]
y_train_df = data['y']

x=X_train_df.to_numpy(dtype=np.float64)
y=y_train_df.to_numpy(dtype=np.float64)

x1_df = X_train_df[X_train_df.index.get_level_values(1) <  '2021-01-01']
x1_np=x1_df.to_numpy(dtype=np.float64)


x2_df = X_train_df[X_train_df.index.get_level_values(1) >= '2021-01-01']
x2_np=x2_df.to_numpy(dtype=np.float64)

y1_df = y_train_df[y_train_df.index.get_level_values(1) <  '2021-01-01']
y1_np = y1_df.to_numpy(dtype=np.float64)

y2_df=y_train_df[y_train_df.index.get_level_values(1)   >= '2021-01-01']
y2_np=y2_df.to_numpy(dtype=np.float64)

In [88]:
X_train_df

open      high       low     close  adjClose      vwap  \
ticker date                                                                     
0050   2010-01-05 -0.687952 -0.688420 -0.688152 -0.688480 -0.686910 -0.688534   
       2010-01-06 -0.688103 -0.687672 -0.687792 -0.687571 -0.686153 -0.688067   
       2010-01-07 -0.687267 -0.687388 -0.687188 -0.687717 -0.686275 -0.687673   
       2010-01-08 -0.687461 -0.687658 -0.687343 -0.687399 -0.686010 -0.687748   
       2010-01-11 -0.687519 -0.687323 -0.687204 -0.687181 -0.685828 -0.687589   
...                     ...       ...       ...       ...       ...       ...   
XSMI   2024-06-24  2.758994  2.784666  2.780434  2.802151  3.380982  2.782671   
       2024-06-25  2.785562  2.766568  2.792536  2.773880  3.348156  2.780705   
       2024-06-26  2.781439  2.766902  2.790117  2.770354  3.344061  2.778274   
       2024-06-27  2.751408  2.737581  2.762153  2.757594  3.329243  2.753243   
       2024-06-28  2.747871  2.730737  2.750774  2.733459  3.301216  2.741772   

                   open_composed  high_composed  low_composed  close_composed  \
ticker date                                                                     
0050   2010-01-05      -0.258004      -0.263814     -0.257305       -0.260749   
       2010-01-06      -0.258044      -0.263636     -0.257056       -0.260265   
       2010-01-07      -0.257671      -0.263533     -0.256978       -0.260577   
       2010-01-08      -0.257922      -0.263867     -0.257266       -0.260663   
       2010-01-11      -0.258031      -0.263707     -0.257002       -0.260414   
...                          ...            ...           ...             ...   
XSMI   2024-06-24       2.339524       2.359556      2.361499        2.374765   
       2024-06-25       2.362142       2.355507      2.378049        2.361970   
       2024-06-26       2.374585       2.359620      2.369213        2.349027   
       2024-06-27       2.361202       2.341013      2.369747        2.350958   
       2024-06-28       2.365084       2.342640      2.347145        2.326038   

                   adjClose_composed  vwap_composed   returns  \
ticker date                                                     
0050   2010-01-05          -0.265271      -0.259975 -0.590986   
       2010-01-06          -0.265076      -0.259758  1.477777   
       2010-01-07          -0.264911      -0.259697 -0.274129   
       2010-01-08          -0.265171      -0.259936  0.484504   
       2010-01-11          -0.265050      -0.259796  0.319309   
...                              ...            ...       ...   
XSMI   2024-06-24           2.676255       2.356167  0.855162   
       2024-06-25           2.661981       2.361688 -0.625792   
       2024-06-26           2.647542       2.360411 -0.109753   
       2024-06-27           2.649696       2.352996 -0.304380   
       2024-06-28           2.621897       2.342557 -0.545867   

                   returns_composed  CDL2CROWS_cumsum  CDL3OUTSIDE_cumsum  \
ticker date                                                                 
0050   2010-01-05         -0.619335          0.237521            0.236719   
       2010-01-06          1.140136          0.237521            0.236719   
       2010-01-07         -0.795143          0.237521            0.236719   
       2010-01-08         -0.251136          0.237521            0.236719   
       2010-01-11          0.562226          0.237521            0.237216   
...                             ...               ...                 ...   
XSMI   2024-06-24          0.681338         -0.799075           -0.715569   
       2024-06-25         -0.405504         -0.799075           -0.715569   
       2024-06-26         -0.411490         -0.799075           -0.715569   
       2024-06-27          0.012047         -0.799075           -0.715569   
       2024-06-28         -0.755222         -0.799075           -0.715569   

                   CDLSHOOTINGSTAR_composed_cumsum  CDLENGULFING_cu

In [54]:
x1_df

open      high       low     close  adjClose      vwap  \
ticker date                                                                     
0050   2010-01-05 -0.687952 -0.688420 -0.688152 -0.688480 -0.686910 -0.688534   
       2010-01-06 -0.688103 -0.687672 -0.687792 -0.687571 -0.686153 -0.688067   
       2010-01-07 -0.687267 -0.687388 -0.687188 -0.687717 -0.686275 -0.687673   
       2010-01-08 -0.687461 -0.687658 -0.687343 -0.687399 -0.686010 -0.687748   
       2010-01-11 -0.687519 -0.687323 -0.687204 -0.687181 -0.685828 -0.687589   
...                     ...       ...       ...       ...       ...       ...   
XSMI   2020-12-25  2.375389  2.376475  2.394477  2.393505  2.685227  2.385882   
       2020-12-28  2.392284  2.406690  2.411476  2.422544  2.716892  2.409197   
       2020-12-29  2.455942  2.446017  2.474386  2.460951  2.758773  2.460260   
       2020-12-30  2.480761  2.471272  2.493658  2.489748  2.790176  2.484814   
       2020-12-31  2.486380  2.476187  2.495424  2.488559  2.788879  2.487598   

                   open_composed  high_composed  low_composed  close_composed  \
ticker date                                                                     
0050   2010-01-05      -0.258004      -0.263814     -0.257305       -0.260749   
       2010-01-06      -0.258044      -0.263636     -0.257056       -0.260265   
       2010-01-07      -0.257671      -0.263533     -0.256978       -0.260577   
       2010-01-08      -0.257922      -0.263867     -0.257266       -0.260663   
       2010-01-11      -0.258031      -0.263707     -0.257002       -0.260414   
...                          ...            ...           ...             ...   
XSMI   2020-12-25       1.952384       1.943082      1.961980        1.951595   
       2020-12-28       1.951278       1.951266      1.971575        1.968849   
       2020-12-29       1.989741       1.986571      2.006750        1.998377   
       2020-12-30       2.009182       1.998489      2.021344        1.994144   
       2020-12-31       2.013310       2.003183      2.023579        2.001405   

                   adjClose_composed  vwap_composed   returns  \
ticker date                                                     
0050   2010-01-05          -0.265271      -0.259975 -0.590986   
       2010-01-06          -0.265076      -0.259758  1.477777   
       2010-01-07          -0.264911      -0.259697 -0.274129   
       2010-01-08          -0.265171      -0.259936  0.484504   
       2010-01-11          -0.265050      -0.259796  0.319309   
...                              ...            ...       ...   
XSMI   2020-12-25           1.979642       1.949963  0.193692   
       2020-12-28           1.997038       1.958434  0.650018   
       2020-12-29           2.027168       1.993016  0.862808   
       2020-12-30           2.022763       2.003423  0.629956   
       2020-12-31           2.030211       2.008005 -0.062786   

                   returns_composed  CDL2CROWS_cumsum  CDL3OUTSIDE_cumsum  \
ticker date                                                                 
0050   2010-01-05         -0.619335          0.237521            0.236719   
       2010-01-06          1.140136          0.237521            0.236719   
       2010-01-07         -0.795143          0.237521            0.236719   
       2010-01-08         -0.251136          0.237521            0.236719   
       2010-01-11          0.562226          0.237521            0.237216   
...                             ...               ...                 ...   
XSMI   2020-12-25          0.151077         -0.799075           -0.707676   
       2020-12-28          0.537585         -0.799075           -0.707676   
       2020-12-29          0.943161         -0.799075           -0.707676   
       2020-12-30         -0.182765         -0.799075           -0.707676   
       2020-12-31          0.196660         -0.799075           -0.707676   

                   CDLSHOOTINGSTAR_composed_cumsum  CDLENGULFING_cu

In [93]:

# x1_df_OBV_composed=x1_df['OBV_composed']
# x1_df_OBV_composed_np=x1_df_OBV_composed.to_numpy(dtype=np.float64)

# x1_df_high=x1_df['high']
# x1_df_high_np=x1_df_high.to_numpy(dtype=np.float64)

x1_df_low_composed=x1_df['low_composed']
x1_df_low_composed_np=x1_df_low_composed.to_numpy(dtype=np.float64)

# x1_df_vwap=x1_df['vwap']
# x1_df_vwap_np=x1_df_vwap.to_numpy(dtype=np.float64)

x1_df_close_composed=x1_df['close_composed']
x1_df_close_composed_np=x1_df_close_composed.to_numpy(dtype=np.float64)


x1_df_adjClose_composed=x1_df['adjClose_composed']
x1_df_adjClose_composed_np=x1_df_adjClose_composed.to_numpy(dtype=np.float64)

# x1_df_returns_composed=x1_df['returns_composed']
# x1_df_returns_composed_np=x1_df_returns_composed.to_numpy(dtype=np.float64)


x1_df_low_composed

ticker  date      
0050    2010-01-05   -0.257305
        2010-01-06   -0.257056
        2010-01-07   -0.256978
        2010-01-08   -0.257266
        2010-01-11   -0.257002
                        ...   
XSMI    2020-12-25    1.961980
        2020-12-28    1.971575
        2020-12-29    2.006750
        2020-12-30    2.021344
        2020-12-31    2.023579
Name: low_composed, Length: 206352, dtype: float64

In [94]:
x1_df['low_composed'].unstack(0)

ticker,0050,0051,069500,069660,0820EA,159901,510050,510180,ANII,BBVAI,BEL,BOVA11,BRAX11,CAC,CRQ,CSSMI,CSSMIM,DBX6,DIA,E40,ECH,EIDO,EWA,EWC,EWD,EWG,EWH,EWJ,EWK,EWL,EWM,EWN,EWP,EWQ,EWS,EWT,EWU,EWW,EWY,EWZ,EXS1.DE,EXS2.DE,EXS3.DE,EZA,FXI,IAEX,IFN,IIF,IJPN,INDY,IQQB,IQQC,ISF,ITWN,IUKD,IWB,IWF,JPXN,JSGI,LYXIB,MDY,MIDD,MXE,NAFTRACISHRS,SFY.AX,SMAL11,SPY,STW.AX,THD,XIT.TO,XIU.TO,XSMI
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2010-01-05,-0.257305,-0.245818,1.834503,2.849253,-0.275213,-0.189624,-0.259925,-0.147761,-0.231831,-0.079089,0.051876,-0.003846,-0.041836,0.248845,0.341492,0.368203,6.483444,-0.180693,0.041989,0.273287,-0.227053,-0.284023,-0.103352,0.294545,-0.198221,0.443706,-0.247397,3.067848,0.178211,7.060694,-0.271988,0.220956,-0.152137,0.486954,-0.211833,-0.244830,-0.018222,-0.259986,1.359541,-0.175536,0.232699,-0.172842,0.098773,-0.145240,-0.212380,0.019414,-0.203542,-0.205588,3.056058,-0.199373,0.181204,-0.212608,-0.025544,-0.244621,-0.154902,7.113859,2.320417,0.848907,0.385378,-0.065576,2.192326,0.131750,-0.276404,-0.249839,-0.105618,0.415198,3.810545,0.142351,-0.267020,-0.241579,0.084553,0.249664
2010-01-06,-0.257056,-0.245634,1.855868,2.869491,-0.275108,-0.201332,-0.259828,-0.146452,-0.231293,-0.079116,0.055114,-0.002058,-0.041122,0.249041,0.346773,0.377780,6.554400,-0.179923,0.043111,0.274862,-0.226079,-0.283954,-0.102345,0.298820,-0.198386,0.446416,-0.247042,3.032470,0.181624,7.140106,-0.271842,0.221789,-0.152004,0.487847,-0.211465,-0.244502,-0.018042,-0.259555,1.381993,-0.174828,0.234901,-0.172317,0.103409,-0.145144,-0.211160,0.018073,-0.202336,-0.204235,3.024958,-0.198783,0.183818,-0.211280,-0.025132,-0.244323,-0.153772,7.155442,2.328810,0.837302,0.379927,-0.066007,2.209127,0.133218,-0.276197,-0.249272,-0.104318,0.443960,3.831580,0.145820,-0.266954,-0.241590,0.087887,0.258085
2010-01-07,-0.256978,-0.245772,1.835310,2.841767,-0.275058,-0.203299,-0.260464,-0.149659,-0.231261,-0.081354,0.051859,-0.005388,-0.042590,0.243833,0.342691,0.372326,6.492468,-0.181502,0.042111,0.268720,-0.225248,-0.283979,-0.103079,0.294124,-0.198626,0.439063,-0.246942,3.007289,0.178335,7.072891,-0.271771,0.217059,-0.153721,0.480309,-0.212316,-0.244501,-0.020114,-0.259517,1.354934,-0.176001,0.230641,-0.172863,0.099142,-0.148560,-0.212375,0.014522,-0.202212,-0.204035,2.998741,-0.198847,0.178663,-0.212732,-0.026991,-0.244306,-0.154312,7.147100,2.322669,0.830646,0.385085,-0.068686,2.208916,0.128662,-0.276260,-0.249227,-0.105105,0.431672,3.832562,0.143983,-0.266954,-0.241390,0.085085,0.253747
2010-01-08,-0.257266,-0.245881,1.849834,2.866172,-0.275060,-0.203809,-0.260806,-0.150799,-0.230699,-0.078212,0.053417,-0.003433,-0.040978,0.249771,0.344253,0.378766,6.462703,-0.179957,0.043780,0.275351,-0.224055,-0.283919,-0.101619,0.294866,-0.197838,0.443603,-0.246766,3.049630,0.180222,7.049542,-0.271792,0.222025,-0.152344,0.488896,-0.212148,-0.244831,-0.017671,-0.259236,1.361301,-0.174682,0.234859,-0.171555,0.102989,-0.146425,-0.212664,0.016534,-0.202421,-0.203983,3.042850,-0.199013,0.181995,-0.213008,-0.024439,-0.244640,-0.152937,7.174707,2.331084,0.841519,0.401063,-0.066518,2.226296,0.133038,-0.276119,-0.248529,-0.103392,0.440595,3.847501,0.148546,-0.266922,-0.241348,0.086145,0.260483
2010-01-11,-0.257002,-0.245394,1.876483,2.916824,-0.274949,-0.203294,-0.260508,-0.148949,-0.230142,-0.073820,0.058399,-0.001707,-0.040181,0.260529,0.350705,0.394779,6.480568,-0.179446,0.043919,0.286580,-0.222027,-0.283841,-0.099003,0.299816,-0.195720,0.458484,-0.246493,3.111351,0.187930,7.082769,-0.271654,0.231294,-0.149489,0.503724,-0.211267,-0.244394,-0.013499,-0.259387,1.382708,-0.174568,0.244569,-0.169216,0.112932,-0.145037,-0.212116,0.021874,-0.201855,-0.203059,3.107180,-0.199117,0.186056,-0.212280,-0.020514,-0.244193,-0.150887,7.210935,2.338901,0.863122,0.392296,-0.061880,2.238589,0.140432,-0.276291,-0.248069,-0.100622,0.418890,3.868892,0.154927,-0.266777,-0.241399,0.089567,0.275804
...,...,...,...,...,...,...,...,...,...,...,...,...,.

##alpha_1

In [95]:
nombre_ticker=len(x1_df.index.get_level_values(0).unique())
print(f'nombre_ticker:{nombre_ticker}')

nombre_date  =len(x1_df.index.get_level_values(1).unique())
print(nombre_date)

def all_function_51(window):
  # 不理解 _l 目的

    def MA2(X):
        if len(X) <= 10:
            return np.ones_like(X)

        # Reshape X into 2D (dates, tickers)
        X = X.reshape(nombre_ticker, nombre_date).T

        # Handle NaN values by replacing NaNs with the column means (or other strategies)
        # Optionally, use np.nan_to_num() if you want to replace NaNs with zeros or some other value
        X = np.nan_to_num(X, nan=np.nan)

        # Use sliding_window_view to create sliding windows for each column (ticker)
        windows = sliding_window_view(X, window, axis=0)

        # Calculate the mean for each window along the axis=1 (which is now the window axis)
        column_means = np.nanmean(windows, axis=2)  # This will compute the mean for each window

        # Initialize result array with NaN for positions where there isn't enough data to compute
        result = np.full_like(X, np.nan, dtype=np.float64)

        # Assign the calculated moving averages to the appropriate positions in the result array
        result[window - 1:, :] = column_means
        result[:window-1,:] = result[window-1,:]
        del windows, column_means
        return result.T.flatten()


    MA2.__name__ = f"MA2_{window}"
    return MA2


def all_function_6(window):
    # @njit(nb.float64[:](nb.float64[:]))
    # def Prod(sr):
    #     n = len(sr)
    #     result = np.empty(n)
    #     if n > 0:
    #         initial_prod = sr[0]  # 这里不使用sr[0]**window以避免对0的任何次方产生不适当的结果
    #         result[:window-1] = initial_prod
    #         for i in range(window - 1, n):
    #             result[i] = np.prod(sr[i - window + 1:i + 1])
    #     else:
    #         result[:] = 1
    #     return result

    ##### 矩阵形式
    def Prod(X):
        if len(X) <= 10:
            return np.ones_like(X)

        # Reshape the input data to 2D (nombre_ticker, nombre_date)
        X = X.reshape(nombre_ticker, nombre_date).T

        # Initialize the result array with NaN values
        result = np.full_like(X, np.nan, dtype=np.float64)

        # Use sliding_window_view to create the sliding windows for each column
        windows = sliding_window_view(X, window, axis=0)

        # Calculate the product for each window and assign it to the result array
        result[window-1:] = np.prod(windows, axis=2)

        # Fill the first `window-1` elements with the initial value (first element in each column)
        result[:window-1] = X[0, :]
        del windows
        return result.T.flatten()

    Prod.__name__ = f"Prod_{window}"
    return Prod

def all_function_47(window):
# checked
    # def ts_corr2(X, Y):
    #     if len(X) <= 10:
    #         return np.ones_like(X)
    #     if len(Y) <= 10:
    #         return np.ones_like(Y)

    #     X = X.reshape(nombre_ticker, nombre_date).T
    #     Y = X.reshape(nombre_ticker, nombre_date).T

    #     res =  np.empty_like(X)
    #     min_len = min(len(X), len(Y))  # 确保 X 和 Y 长度一致
    #     for i in range(min_len - window + 1):
    #         X_ = X[i:i+window]
    #         Y_ = Y[i:i+window]
    #         valid_data_x = []
    #         valid_data_y = []
    #         # for idx in range(len(X_)):
    #         #     if not np.isnan(X_[idx]) and not np.isnan(Y_[idx]):
    #         #         valid_data_x.append(X_[idx])
    #         #         valid_data_y.append(Y_[idx])

    #         ##### 加速
    #         # Masking NaNs to get valid data
    #         valid_mask = ~np.isnan(X_) & ~np.isnan(Y_)
    #         valid_data_x = X_[valid_mask]
    #         valid_data_y = Y_[valid_mask]

    #         if len(valid_data_x) > 2:  # 确保有足够的数据点进行相关系数计算
    #             res[i+window-1] = np.corrcoef(valid_data_x, valid_data_y)[0, 1]
    #     return res.T.flatten()


    def ts_corr2(X, Y):
        """
        Calculates the ts_corr2 factor in 2D format.

        Args:
            X: First input data as a 1D numpy array.
            Y: Second input data as a 1D numpy array.
            window: Window size for calculation.
            nombre_ticker: Number of tickers.
            nombre_date: Number of dates.

        Returns:
            The ts_corr2 factor in 2D format as a numpy array.
        """
        if len(X) <= 10 or len(Y) <= 10:
            return np.ones_like(X)

        # Reshape X and Y to 2D
        X = X.reshape(nombre_ticker, nombre_date).T  # Reshape to [date, ticker]
        Y = Y.reshape(nombre_ticker, nombre_date).T  # Reshape to [date, ticker]

        # Create sliding windows for X and Y
        X_windows = sliding_window_view(X, window,0)
        Y_windows = sliding_window_view(Y, window,0)

        # Calculate means for window data
        X_mean = np.mean(X_windows, axis=2, keepdims=True)
        Y_mean = np.mean(Y_windows, axis=2, keepdims=True)

        # Calculate covariance (numerator)
        numerator = np.sum((X_windows - X_mean) * (Y_windows - Y_mean), axis=2)

        # Calculate variances (denominators)
        X_variance = np.sum((X_windows - X_mean) ** 2, axis=2)
        Y_variance = np.sum((Y_windows - Y_mean) ** 2, axis=2)

        # Calculate correlation using covariance / sqrt(variance_x * variance_y)
        denominator = np.sqrt(X_variance * Y_variance)
        denominator = np.where(denominator > 1e-10, denominator, 1e-10)  # Avoid division by zero

        # Calculate correlation
        correlation = numerator / denominator

        # Initialize result matrix
        result = np.full_like(X, np.nan, dtype=np.float64)
        result[window - 1:, :] = correlation  # Populate the result after the first full window
        result[:window-1,:] = result[window-1,:]
        del X_windows, Y_windows, X_mean, Y_mean, numerator, X_variance, Y_variance, denominator, correlation
        return result.T.flatten()  # Flatten back to 1D

    ts_corr2.__name__ = f"ts_corr2_{window}"
    return ts_corr2



nombre_ticker:72
2866


In [96]:
window=200
window_list = [i for i in range(2, window)]


In [97]:
# alpha_2	-0.030341756658082676	MA2_174(Prod_7(low_composed, close_composed), MEAN_SECTION_7(adjClose_composed))	2	6
# alpha_1	-0.20837128141117625	Count_10(ts_corr2_24(NEUTRALIZE_SECTION_3(ts_argmax_22(ts_freq2_16(Count_7(low_composed)))), MIDPOINT2_9(MA2_17(NEUTRALIZE_SECTION_18(LINEARREG_INTERCEPT_18(adjClose_composed))))))	6	12


dict_functions = {f"MA2_{i}": all_function_51(i) for i in window_list}
MA2_61 = dict_functions["MA2_61"]


dict_functions = {f"Prod_{i}": all_function_6(i) for i in window_list}
Prod_7 = dict_functions["Prod_16"]

dict_functions = {f"ts_corr2_{i}": all_function_47(i) for i in window_list}
ts_corr2_7= dict_functions["ts_corr2_45"]

In [98]:



alpha_2=	MA2_61(MA2_61(ts_corr2_7(x1_df_adjClose_composed_np,x1_df_close_composed_np)))
x1_df['alpha_2']=alpha_2

x1_df=x1_df[['alpha_2']]
x1_df

# # dict_functions = {f"delay_{i}": all_function_1(i) for i in window_list}
# # delay_functions, delay_names = functions_name_arity1(dict_functions)

<ipython-input-98-4c841191e121>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x1_df['alpha_2']=alpha_2


alpha_2
ticker date                
0050   2010-01-05  0.988585
       2010-01-06  0.988585
       2010-01-07  0.988585
       2010-01-08  0.988585
       2010-01-11  0.988585
...                     ...
XSMI   2020-12-25  0.999202
       2020-12-28  0.999178
       2020-12-29  0.999154
       2020-12-30  0.999130
       2020-12-31  0.999106

[206352 rows x 1 columns]

In [201]:
# 初始化一个空字典来存储结果
value_dict_new = {}

window = 500 # 举个例子
num_rows_to_delete = window * 5

# 对每个因子进行迭代
for factor_name in x1_df.columns:
    # 对每个因子进行解堆叠操作，ticker 从索引转为列
    factor_df = x1_df[factor_name].unstack(level=0) #容易报错，注意应该以哪个轴去 pivot
    factor_df = factor_df.iloc[num_rows_to_delete:]
    # 存储转换后的 DataFrame 到字典
    value_dict_new[factor_name] = factor_df
value_dict_new

{'alpha_2': ticker          0050      0051    069500    069660    0820EA    159901  \
 date                                                                     
 2019-08-07  0.999730  0.999866  0.999901  0.999910  0.980907  0.999900   
 2019-08-08  0.999719  0.999860  0.999916  0.999921  0.980389  0.999896   
 2019-08-09  0.999707  0.999853  0.999927  0.999929  0.979873  0.999893   
 2019-08-12  0.999692  0.999843  0.999934  0.999935  0.979358  0.999889   
 2019-08-13  0.999675  0.999832  0.999939  0.999939  0.978846  0.999885   
 ...              ...       ...       ...       ...       ...       ...   
 2020-12-25  0.992983  0.994413  0.999722  0.999858  0.976536  0.999986   
 2020-12-28  0.993167  0.994475  0.999720  0.999858  0.976336  0.999986   
 2020-12-29  0.993353  0.994538  0.999719  0.999857  0.976164  0.999986   
 2020-12-30  0.993539  0.994604  0.999717  0.999857  0.976027  0.999986   
 2020-12-31  0.993726  0.994673  0.999714  0.999856  0.975924  0.999986   
 
 ticker     

In [150]:
# file_path_1="/content/drive/MyDrive/Colab/data_clean_合集_最新.pkl"
# data_clean=pd.read_pickle(file_path_1)
# base_Factor = data_clean.columns.tolist()
# data_clean=pd.read_pickle(file_path_1)
# data_clean= data_clean[data_clean.index.get_level_values(1) <  '2021-01-01']
# data_clean

In [151]:
def Return_N_IC_train(df,y,n,name = '',Rank_IC = True, window=30):
    # file_path_1="/content/drive/MyDrive/Colab/data_clean_合集_最新.pkl"
    # data_clean=pd.read_pickle(file_path_1)
    # base_Factor = data_clean.columns.tolist()
    # data_clean=pd.read_pickle(file_path_1)
    # data_clean= data_clean[data_clean.index.get_level_values(1) <  '2021-01-01']
    # # window = 200 # 举个例子
    num_rows_to_delete = window * 5
    # # file_path_2='/content/drive/MyDrive/Colab/full_etf_with_ind_no_fill.pkl'
    # # data=pd.read_pickle(file_path_2)
    # # data= data[data.index.get_level_values(1) <  '2021-01-01']

    # # price=data[['close']]
    # # data_clean['y'] = price['close'].unstack('ticker').pct_change(5).shift(-6).unstack()
    # # num_rows_to_delete = window * 5

    # # data_clean.replace([0.0,0], np.nan, inplace=True)
    # # data_clean.interpolate(method='linear', inplace=True)
    # data_clean.fillna(method='ffill', inplace=True)  # 向前填充
    # data_clean.fillna(method='bfill', inplace=True)
    # data = data_clean
    # data.replace(0, np.nan, inplace=True)
    return_n=y.unstack().T
    return_n = return_n.iloc[num_rows_to_delete:]
    print(len(return_n),len(df))
    x = df.corrwith(return_n,axis =1,method='spearman').dropna(how = 'all')
    t_stat,_ = stats.ttest_1samp(x, 0)

    IC = {'name'   :name,
          'IC mean':round(x.mean(),4),
          'IR'     :round(x.mean()/x.std(),4),
          'IC_IR'  :round(x.mean() / x.std() * math.sqrt(250), 4),
          't_stat' :round(t_stat,4)}
    IC = pd.DataFrame([IC])
    return x,IC

##Train结果

In [152]:
import math
from tqdm import tqdm
ic_df      = pd.DataFrame()
ic_summary = pd.DataFrame()

for key,value in tqdm(value_dict_new.items()):
    ic, ic_summary_temp = Return_N_IC_train(value_dict_new[key],y1_df,5,key,window=500)
    ic_df = pd.concat([ic_df, ic.to_frame(key)], axis=1)
    ic_summary = pd.concat([ic_summary,ic_summary_temp.set_index('name')],axis = 0)

ic_summary.sort_values(by = 'IR')

ic_summary=ic_summary.rename(columns={'IC>0':'IC_0'})
ic_summary.sort_values(by='IC mean', ascending=False, inplace=True)
ic_summary=ic_summary.rename(columns={'IC mean':'IC_mean'})
ic_summary



  0%|          | 0/1 [00:00<?, ?it/s]

366 366


100%|██████████| 1/1 [00:00<00:00,  1.85it/s]


,IC_mean,IR,IC_IR,t_stat
name,,,,
alpha_2,0.0582,0.3061,4.8406,5.857


In [ ]:
import matplotlib.pyplot as plt

groups = x1_df.groupby('ticker')

for key,group in groups:
    # 获取每个组的数据

    group.loc[:,'alpha_2'].plot()
    plt.show()
    plt.close()